# Preprocessing Document

credit: Spasic et al. (2019)  
https://github.com/dokato/c2s2

In [1]:
import numpy as np
import re
import csv
import os
#from nltk.corpus import stopwords
import spacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
import mysql.connector
from nltk.stem import PorterStemmer
import regex
from collections import Counter
import datetime
from datetime import datetime, date
from pymetamap import MetaMap
import timeout_decorator
import math
import nltk
from nltk.corpus import stopwords
import requests
import json
import pandas as pd

In [2]:
cnx = mysql.connector.connect(host="localhost", user="root", password="1q1q1q1q", database = 'umls16ab')
cnx.start_transaction(isolation_level='READ COMMITTED')
mycursor = cnx.cursor()

mm = MetaMap.get_instance('/Users/silve/Documents/UMLS/MetaMap/public_mm/bin/metamap')

In [3]:
nlp = spacy.load('en_core_web_sm')
ps =PorterStemmer()

In [4]:
def listize(doc):
    pdoc = [cr for cr in doc.split(' .')]
    pdoc = [cr.replace('\n', ' ') for cr in pdoc if 'TITLE' not in cr]
    #pdoc = [cr for cr in pdoc if cr != ' ']
    pdoc = [cr.replace('XXX', '') for cr in pdoc]
    pdoc = [cr+' ' for cr in pdoc]
    pdoc = [cr.replace('  ', ' ') for cr in pdoc]
    pdoc = [cr for cr in pdoc if cr != '  ']
    pdoc = [cr for cr in pdoc if cr != ' ']
    pdoc = [cr for cr in pdoc if cr != '']
    pdoc = [cr+'.' for cr in pdoc]

    return pdoc

In [5]:
def find_between(s, first, last):
    try:
        start = s.rindex(first) + len(first)
        end = s.rindex(last, start)
        return s[start:end]
    except ValueError:
        return ""

In [6]:
def preproc(doc):
    # --- dictionaries
    dic1 = dict(csv.reader(open('./lexicon/abbrev.txt')))
    dic2 = dict(csv.reader(open('./lexicon/mwe.txt')))
    #dic3 = dict(csv.reader(open('./lexicon/brand.txt')))

    # Remove line breakder if there are multiple lines in a paragraph and each line holds more than 50 characters 
    for line in re.findall('.{60,}\n{1,3}', doc):
        if not line.strip().endswith('.'):
            doc = doc.replace(line, line.strip()+" ")
    
    # --- gender and age
    doc = doc.replace('gentleman', 'male')
    doc = doc.replace(' M ', ' male ') ### added by me
    doc = re.sub('\nM\n', '\nmale\n', doc) ### added by me
    doc = re.sub('\sman\s', ' male ', doc) ### added by me
    doc = re.sub('\smen\s', ' male ', doc) ### added by me
    doc = doc.replace('lady', 'female')
    doc = doc.replace(' F ', ' female ') ### added by me
    doc = re.sub('\nF\n', '\nfemale\n', doc) ### added by me
    doc = re.sub('woman\s', 'female ', doc)
    doc = re.sub('women\s', 'female ', doc) ### added by me
    doc = re.sub(' y/*o male ', ' year old male ', doc, flags=re.IGNORECASE)
    doc = re.sub(' y/*o m ', ' year old male ', doc, flags=re.IGNORECASE)
    doc = re.sub(' y/*o female ', ' year old female ', doc, flags=re.IGNORECASE)
    doc = re.sub(' y/*o f ', ' year old female ', doc, flags=re.IGNORECASE)
    for age in re.findall(' (\d+) ([fe]*male)', doc): ### added by me
        doc = re.sub(' (\d+) ([fe]*male)', ' \\1 year old \\2', doc)

    # --- titles so that they don't get confused with other acronyms
    doc = doc.replace('M.Sc.', '')
    doc = doc.replace('Ph.D.', '')
    doc = doc.replace('PhD', '')
    doc = doc.replace('Jr.', '')
    doc = doc.replace('JR.', '')
    doc = doc.replace('Mr.', 'male')
    doc = doc.replace('Ms.', 'female')
    doc = doc.replace('Mrs.', 'female')

    # --- enclytics
    doc = doc.replace('aren\'t', 'are not')
    doc = doc.replace('can\'t', 'cannot')
    doc = doc.replace('couldn\'t', 'could not')
    doc = doc.replace('didn\'t', 'did not')
    doc = doc.replace('doesn\'t', 'does not')
    doc = doc.replace('don\'t', 'do not')
    doc = doc.replace('hadn\'t', 'had not')
    doc = doc.replace('hasn\'t', 'has not')
    doc = doc.replace('haven\'t', 'have not')
    doc = doc.replace('isn\'t', 'is not')
    doc = doc.replace('wasn\'t', 'was not')
    doc = doc.replace('wouldn\'t', 'would not')
    doc = doc.replace('won\'t', 'will not')
    doc = doc.replace('con\'t', 'continue')
    doc = doc.replace('cont\'d', 'continue')
    doc = doc.replace('\'m', ' am')
    doc = doc.replace('\'ll', ' will')
    doc = doc.replace('\'s', '')
    doc = doc.replace('\'ve', '')
    doc = doc.replace('\'d', '')
    doc = doc.replace('\'ed', '')

    # --- special punctuation
    doc = re.sub('\sC\\.', ' C ', doc, flags=re.IGNORECASE) # C. diff --> C diff
    doc = re.sub('\sE\\.', ' E ', doc, flags=re.IGNORECASE) # E. coli --> E coli
    doc = re.sub('[^\w]vit\\.', ' vit ', doc, flags=re.IGNORECASE) # vit. D --> vit D
    doc = doc.replace('x. ', 'x ') # fx. of --> fx of
    doc = re.sub('\sq\\.\s', ' q.', doc, flags=re.IGNORECASE) # q. a.m. --> q.a.m.
    doc = re.sub('([a-z])\\.([a-z])\\.([a-z])\\.', '\\1\\2\\3', doc, flags=re.IGNORECASE) # q.a.m. --> qam
    doc = re.sub('([a-z])\\.([a-z])\\.', '\\1\\2', doc, flags=re.IGNORECASE) # q.d. --> qd
    #doc = re.sub('q\\.(\d)', 'q\\1', doc, flags=re.IGNORECASE) # q.4 --> q4
    doc = re.sub('\sq\\.\s', ' q ', doc, flags=re.IGNORECASE|re.S) # q.\n --> q
    doc = re.sub('\sq\\.', ' q ', doc, flags=re.IGNORECASE) # q. --> q
    doc = re.sub('\sq\s+', ' q ', doc, flags=re.IGNORECASE|re.S) # q\n --> q
    doc = re.sub('\sh\\.\s', ' h ', doc, flags=re.IGNORECASE) # h. --> h
    doc = re.sub('(\\.)([a-z])', '. \\2', doc, flags=re.IGNORECASE) # .blah --> . blah
    doc = doc.replace('.)', '. ') # overzealous numbering
    doc = doc.replace('ELEM.', 'ELEM ') # elemental
    doc = doc.replace('TAB.SR', 'TABLET')
    doc = re.sub('\(\s*S\s*\)', ' ', doc, flags=re.IGNORECASE)
    
    # --- record date
    #doc = re.sub('Record date: (\d\d\d\d)-(\d\d)-(\d\d)', '. This is record date \\1\\2\\3 .', doc)

    # --- dates
    doc = re.sub('\s\d\d/\d\d/\d\d\d\d\s', ' ', doc)
    
    # ambiguous abbreviations
    doc = re.sub('(MI),* *\d{5}', 'Michigan', doc)
    doc = re.sub('(MI),* *null', 'Michigan', doc)
    doc = re.sub('(CA),* *\d{5}', 'California', doc)
    doc = re.sub('(CA),* *null', 'California', doc)
    doc = re.sub('(TX),* *\d{5}', 'Texas', doc)
    doc = re.sub('(TX),* *null', 'Texas', doc)

    # --- force sentence spliting where punctuation is not used properly
    doc = re.sub('\n[0-9]+\\.\s', ' . This is a list item. ', doc) # 1. blah blah --> . This is a list item. blah blah
    doc = re.sub('\n[0-9]+\)\s', ' . This is a list item. ', doc)  # 1) blah blah --> . This is a list item. blah blah
    doc = re.sub('\n-+', ' . This is a list item. ', doc)          # -- blah blah --> . This is a list item. blah blah
    doc = re.sub('\t', ' ', doc) ### added by me
    doc = re.sub('creatinine\s*:\s*', 'creatinine ', doc, flags=re.IGNORECASE)
    doc = re.sub('creat\s+', 'creatinine', doc, flags=re.IGNORECASE)
    doc = re.sub('[\s/]PE:', '\nPHYSICAL EXAMINATION:', doc)
    doc = re.sub('\n([^\n]+): ', '\n . \\1: ', doc)
    #doc = re.sub('\n+', ' ', doc)
    doc = re.sub('_+', '\n', doc) ### modified by me
    doc = re.sub('--+', ' . ', doc)
    doc = re.sub('==+', ' . ', doc)
    doc = re.sub('##+', ' ', doc)
    doc = re.sub('-', ' ', doc)
    doc = re.sub('\\*+', '.', doc)
    doc = re.sub('\\"', '', doc)
    #doc = re.sub('\s+', ' ', doc)
    doc = re.sub('~', ' ', doc)
    doc = re.sub('\?', '', doc) ### added by me
    doc = re.sub('(\d)([a-z])', '\\1 \\2', doc, flags=re.IGNORECASE) # 1unit -> 1 unit
    #    doc = re.sub('(\d)\-([a-zA-Z])', '\\1 - \\2', doc)
    doc = re.sub(' but ', ' \. but ', doc, flags=re.IGNORECASE) # blah but blah --> blah . but blah ### added by me
    doc = re.sub(' so ', ' \. so ', doc, flags=re.IGNORECASE) # blah so blah --> blah . so blah ### added by me
        
    # medical terminologies 
    doc = re.sub('[aA]\-fib ', 'atrial fibrillation ', doc)
    doc = re.sub('[aA]\s*fib ', 'atrial fibrillation ', doc)
    doc = re.sub('[aA]\\.\s*fib ', 'atrial fibrillation ', doc)
    doc = re.sub('[vV]\-fib', 'ventricular fibrillation', doc)
    doc = re.sub('[vV]\s*fib', 'ventricular fibrillation', doc)
    doc = re.sub('[vV]\\.\s*fib', 'ventricular fibrillation', doc)
    doc = re.sub(' DM\s*1 ', ' type 1 diabetes mellitus ', doc, flags=re.IGNORECASE)
    doc = re.sub(' DM\s*2 ', ' type 2 diabetes mellitus ', doc, flags=re.IGNORECASE)
    doc = re.sub('(?<=[A-Za-z])\sca\.? ', ' cancer ', doc, flags=re.IGNORECASE) # breast ca. --> breast cancer
    doc = re.sub('(?<=\d)\su\.? ', ' unit ', doc, flags=re.IGNORECASE) # 2 u. --> 2 unit
    doc = re.sub('\shb\s?a1\sc\s', ' glycated hemoglobin-a1c ', doc, flags=re.IGNORECASE) # hba1c --> glycated hemoglobin-a1c
    doc = re.sub('\shea?moglobin\sa1\sc\s', ' glycated hemoglobin-a1c ', doc, flags=re.IGNORECASE) # hemoglobin a1c --> glycated hemoglobin-a1c
    doc = re.sub('\shb([\s\\.,])', ' hemoglobin\\1 ', doc, flags=re.IGNORECASE) # hb --> hemoglobin
    doc = re.sub('\s5\s?fu\s', ' fluorouracil ', doc, flags=re.IGNORECASE) # 5fu --> fluorouracil
    doc = re.sub('[xX] ray', 'x-ray', doc)
    doc = re.sub('chest[^\n\.]*pa ', 'radiology of chest ', doc, flags=re.IGNORECASE) ### added by me
    doc = re.sub('chest x-ray', 'radiology of chest', doc, flags=re.IGNORECASE) ### added by me
    doc = re.sub('(nkda)\\.', '\\1', doc, flags=re.IGNORECASE)
    doc = re.sub('(nka)\\.', '\\1', doc, flags=re.IGNORECASE)
    doc = re.sub('\sb\s6\s', ' pyridoxine ', doc, flags=re.IGNORECASE|re.S)
    doc = re.sub('\sb\s12\s', ' cobalamin ', doc, flags=re.IGNORECASE|re.S)
    doc = re.sub('\sb6\/b12\s', ' pyridoxine cobalamin ', doc, flags=re.IGNORECASE|re.S)
    doc = re.sub('\sb12\/b6\s', ' pyridoxine cobalamin ', doc, flags=re.IGNORECASE|re.S)
    
    #doc = re.sub('\s+', ' ', doc)
    #doc = re.sub(' \.', ' .\n', doc)
    
    #doc = re.sub('[0-9]+', '%%%', doc) # erase numbers. ### Added by me
    #doc = re.sub(' in ', ' ', doc) # erase 'in' ##3 Added by me
    #doc = re.sub(' of ', ' ', doc) # erase 'of' ### Added by me
    #doc = re.sub(' and ', ' ', doc) # erase 'and' ### Added by me
    #doc = re.sub(' or ', ' ', doc) # erase 'or' ### Added by me
    #doc = re.sub(' is ', ' ', doc) # erase 'is' ### Added by me
    
    # sides and locations
    doc = re.sub('\sR\\.*\s', ' right ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sRt\\.*\s', ' right ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sL\\.*\s', ' left ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sLt\\.*\s', ' left ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sp\\.*\s*o\\.*\s', ' oral ', doc, flags=re.IGNORECASE)
    
    # abbreviations
    doc = re.sub('\ss\/p\s', ' status post ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sc\s*\/*\s*c\s', ' chief complaint ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sc\s*\/\s*o\s', ' complaining of ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sc\s*\/\s*b\s', ' complicated by ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sc\s*\/\s*w\s', ' continue with ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sf\s*\/\s*h\s', ' family history ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sn\s*\/\s*v\s', ' nausea vomiting ', doc, flags=re.IGNORECASE)
    doc = re.sub('\so\s*\/\s*e\s', ' on examination ', doc, flags=re.IGNORECASE)
    doc = re.sub('\se\s*\/\s*o\s', ' evidence of ', doc, flags=re.IGNORECASE)
    doc = re.sub('\snc\s*\/\s*at\s', ' normocephalic atraumatic ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sat\s*\/\s*nc\s', ' normocephalic atraumatic ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sd\s*\/\s*c\s', ' discontinued ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sd\s*\/\s*o\s', ' disorder ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sf\s*\/\s*up\s', ' follow up ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sf\s*\/\s*u\s', ' follow up ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sh\s*\/\s*o\s', ' history of ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sy\s*\/*\s*o\s', ' year old ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sr\s*\/\s*o\s', ' rule out ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sw\s*\/\s*u\s', ' work up ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sw\s*\/\s*o[ut]*\s', ' with out ', doc, flags=re.IGNORECASE)
    doc = re.sub('\sw\s*\/\s', ' with ', doc, flags=re.IGNORECASE)
    #doc = re.sub(' history of ', ' ', doc, flags=re.IGNORECASE) ### Added by me
    
    # line breakers
    doc = re.sub('(\\. )*\\.', '.', doc)
    doc = re.sub('([A-Za-z0-9]):(\s)+', '\\1 :\\2', doc, flags=re.IGNORECASE) # blah: --> blah : ### added by me
    doc = re.sub('([^\\.:])(\n)([^\\.:])', '\\1.\n\\3', doc, flags=re.IGNORECASE) # blah\nblah --> blah.\nblah ### added by me
    doc = re.sub('([A-Za-z0-9])(\\.)\n', '\\1 .\n', doc, flags=re.IGNORECASE) # blah. --> blah .
    
    # Special characters
    doc = re.sub('\(', '', doc) # remove parenthesis. ### added by me
    doc = re.sub('\)', '', doc) # remove parenthesis, ### added by me
    doc = doc.replace(']', '') # remove square brackets. ### added by me
    doc = doc.replace('[', '') # remove square brackets. ### added by me
    #doc = doc.replace('/', '') # remove slashes ### added by me
    doc = re.sub(r'\\', ' ', doc) # remove back slash### added by me
    
    # --- parse document
    parsedDoc = nlp(doc)

    sents = []
    # the "sents" property returns spans
    # spans have indices into the original string
    # where each index value represents a token
    for span in parsedDoc.sents:
        # go from the start to the end of each span, returning each token in the sentence
        # combine each token using join()
        sent = ''.join(parsedDoc[i].string for i in range(span.start, span.end)).strip()
        sents.append(sent)
    
    doc = ' '
    for sentence in sents: 
        # sentence = re.sub('\\: \\.', ':', sentence)
        if sentence not in ['This is a list item.', '.']:
            parse = nlp(sentence)
            tokenized = ' '
            for i, token in enumerate(parse):
                t = token.lower_
                if t in dic1.keys(): 
                    t = dic1[t]
                tokenized += t + ' '             
            for key in dic2: 
                tokenized = tokenized.replace(' ' + key + ' ',  ' ' + dic2[key] + ' ')
                tokenized = tokenized.strip(' ')
            #for stopword in stopwords.words("english"):
            #    tokenized = tokenized.replace(' '+stopword+' ', ' ')
            #    tokenized = tokenized.strip(' ')
            #for brand in dic3: 
            #    tokenized = tokenized.replace(' ' + brand + ' ',  ' ' + dic3[brand] + ' ')
            #    tokenized = tokenized.strip(' ')
            if tokenized not in ['']:
                doc += tokenized + '\n '
    
    # Configure section headers by putting ":" at the end of them
    #doc = re.sub('family history .\n', 'family history :\n', doc)
    #doc = re.sub('family history : ', 'family history :\n', doc)
    #doc = re.sub('social history .\n', 'social history :\n', doc)
    #doc = re.sub('social history : ', 'social history :\n', doc)
    
    # To divide doc by setions. ### added by me
    doc = re.sub('\n', ' ', doc)
    doc = re.sub(' \. ', ' * ', doc)
    doc = re.sub('  ', ' ', doc)
    doc = re.sub('\* ([A-Za-z ]+) :', '.\n \\1 :\n', doc)
    doc = re.sub(' \*', ' .', doc)
    doc = re.sub('\n ', '\n', doc)
    
    for sectionHeader in re.findall('[A-Za-z ]+ :\n', doc):
        #print(sectionHeader)
        if not ('history' in sectionHeader or 'exam' in sectionHeader or 'lab' in sectionHeader 
                or 'med' in sectionHeader or 'allerg' in sectionHeader 
                or 'plan' in sectionHeader):
            #print(sectionHeader)
            nonSectionHeader = sectionHeader.replace('\n', ' ')
            doc = re.sub(sectionHeader, nonSectionHeader, doc)
    
    # normalize age expression
    for age in re.findall(' (\d+) *y ', doc): ### added by me
        doc = re.sub(' (\d+) *y ', '\\1 year old', doc)
    
    # --- remove allergy information
    doc = re.sub('(no known drug allergies)', '\\1 .\n', doc)
    doc = re.sub('(no known allergies)', '\\1 .\n', doc)
    doc = re.sub(' all :', ' allergies :', doc)
    doc = re.sub('( allerg[^\n]+:)', ' allergies :', doc)
    doc = re.sub('\n *allergies *\n', ' allergies : ', doc, flags=re.S)
    doc = re.sub(' allergies ?\/ \n', ' allergies : ', doc, flags=re.S)
    doc = re.sub('(allergies : )\n', '\\1', doc, flags=re.S)
    doc = re.sub('(adverse reactions? : )\n', '\\1', doc, flags=re.S)
    doc = re.sub(' allergies : .\n ([A-Za-z0-9/\s\n,;]+) .\n ([A-Za-z0-9]+) : .\n', ' XXX .\n \\2 : .\n', doc) ### added by me   
    for item in re.findall('\s*allerg[^\\.]+', doc): ### modified by me
        doc = re.sub(item, ' XXX ', doc)
    for item in re.findall(' allerg[^\n\\.]+', doc):
        doc = re.sub(' allerg[^\n\\.]+', ' XXX ', doc)
    for item in re.findall('adverse reaction[^\n\\.]+', doc):
        doc = re.sub(' adverse reaction[^\n\\.]+', ' XXX ', doc)
                           
    # --- remove information about family members
    for item in re.findall('family history :\n[^\n\.]+ ', doc): 
        doc = re.sub(item, 'XXX ', doc, flags=re.S)
    for item in re.findall('family history \w[^\n\\.]+ ', doc): 
        doc = re.sub('family history \w[^\n\\.]+', 'XXX ', doc)
    for item in re.findall('family history : \w[^\n\\.]+ ', doc): # added by me
        doc = re.sub('family history : \w[^\n\\.]+ ', 'XXX ', doc, flags=re.S)
    for item in re.findall('family history \. [^\n\\.]+ ', doc): # added by me
        doc = re.sub('family history \. [^\n\\.]+ ', 'XXX ', doc, flags=re.S)

    for item in re.findall('[^\n\\.:;]+family member[^\n\\.]+', doc): 
        doc = re.sub('[^\n\\.:;]+family member[^\n\\.]+', 'XXX ', doc)

    for item in re.findall('[^\n\\.:;]+died[^\n\\.]+', doc): 
        doc = re.sub('[^\n\\.:;]+died[^\n\\.]+', 'XXX ', doc)
    for item in re.findall('[^\n\\.:;]+death[^\n\\.]+', doc):
        doc = re.sub('[^\n\\.:;]+death[^\n\\.]+', 'XXX ', doc)
    for item in re.findall('[^\n\\.:;]+passed away[^\n\\.]+', doc): 
        doc = re.sub('[^\n\\.:;]+passed away[^\n\\.]+', 'XXX ', doc)
    
    # -- remove redundancy
    #doc = re.sub(' status post ', ' ', doc) ### added by me # remove 's/p'
    doc = re.sub(' scan ', ' ', doc) ### added by me # remove 'scan'
    
    doc = re.sub(' ,\s*\n ', ' , ', doc) ### added by me 
    doc = re.sub('(?<=[A-Za-z0-9/])\n', ' . \n', doc) ### added by me
    #doc = re.sub('(?<=[A-Za-z0-9/])\n', ' ', doc, flags=re.IGNORECASE) ### added by me
    #doc = re.sub('\n[A-Za-z0-9 ]+ :\n', '\nTITLE .\n', doc) ### added by me
    #doc = re.sub(' : .\n ', ' ', doc) ### added by me
    doc = re.sub(' {2,3}', ' ', doc) ### added by me
    doc = re.sub('( \.){2,}', '\\1', doc) ### added by me
    doc = re.sub(' {2,}', ' ', doc) ### added by me
    
    # -- correct ordinal numbers
    #doc = re.sub('(?<=[^\.])1 st ', '1st ', doc) ### added by me
    #doc = re.sub('(?<=[^\.])2 nd ', '2nd ', doc) ### added by me
    #doc = re.sub('(?<=[^\.])3 rd ', '3rd ', doc) ### added by me
    #doc = re.sub('([4-9]) th ', '\\1th ', doc) ### added by me
    
    #doc = listize(doc)
    
    return doc

In [7]:
def to_aui(cui):
    sql = "select distinct aui from mrconso where cui = %s and tty = %s"
    val = (cui, 'pt')
    mycursor.execute(sql, val)
    result = mycursor.fetchall()
    return result

In [8]:
def to_str(aui):
    sql = 'select distinct str from mrconso where aui = %s and tty = %s'
    val = (aui, 'pt')
    mycursor.execute(sql, val)
    result = mycursor.fetchall()
    if result:
        return result[0][0].replace(' ', '_')
    else:
        return None
    #return result[0][0]

In [9]:
def from_fsn_to_aui(term):
    sql = 'select distinct cui from mrconso where str = %s and sab = %s'
    val = (term, 'SNOMEDCT_US')
    mycursor.execute(sql, val)
    result = mycursor.fetchall()
    
    if result:
        cui =  result[0][0]
        aui = to_aui(cui)[0][0]
        if aui:
            return aui
        else:
            return None
    else:
        return None
    

In [10]:
def show_ptr_str(aui): # returns (aui, str) of parents of the concept (in aui)
    sql = "select distinct ptr from mrhier where aui = %s and rela=%s"
    val = (aui, 'isa')
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()

    listOfList = []
    for i in range(len(myresult)):
        auiList = [a for a in myresult[i][0].split('.')]
        auiList.append(aui)
        listOfList.append(auiList)
    
    results = []
    for lis in listOfList:
        result = []
        for i in lis:
            sql2 = "select distinct aui,str from mrconso where aui = %s and tty = %s"
            val2 = (i, "pt")
            mycursor.execute(sql2, val2)
            result2 = mycursor.fetchall()
            if result2:
                result.append(result2[0])
            else:
                pass
        results.append(result)
            
    return results
    

In [11]:
def show_ptr(aui): # returns (aui, str) of parents of the concept (in aui)
    sql = "select distinct ptr from mrhier where aui = %s and rela=%s"
    val = (aui, 'isa')
    mycursor.execute(sql, val)
    myresult = mycursor.fetchall()

    listOfList = []
    for i in range(len(myresult)):
        auiList = [a for a in myresult[i][0].split('.')]
        auiList.append(aui)
        listOfList.append(auiList)
    return listOfList

In [12]:
def determine_wt(aui): # v.1.2
    ## Categories
    # nil 
    # sex
    # preg: pregnant
    # dis: diseases / abnormal findings
    # sub: substance / devices
    # med: medication
    # proc: procedure, either for evaluation or treatment {mod: , loc: , side: , temp: }
    # loc: anatomical or geographical sites (attribute)
    # side: Left or Right (attribute)
    # temp: temporality (attribute)
    # neg: negations (attribute)
    # desc: general descriptors, which do not have attributes. Can be a value of an attribute
    
    dedup_list = []
    for lst in show_ptr(aui):
        for a in lst:
            if a not in dedup_list:
                dedup_list.append(a)
                
    dedup_str_list = []
    for lst in show_ptr(aui):
        for a in lst:
            if a not in dedup_str_list:
                dedup_str_list.append(a.lower())
                
    dedup_token_list = []
    for s in dedup_str_list:
        for t in s.split(' '):
            if t not in dedup_token_list:
                dedup_token_list.append(t)
    
    if 'finding' in to_str(aui):
        wt = 0.00
        cat = ('nil', 'finding')
    
    # OBSERVABLE ENTITY ****************************************************************************************
    # is a question
    
    elif 'A3241965' in dedup_list: # Activity of daily living (Level 4) >> Walking
        wt = 0.40
        cat = ('desc', 'Activity of daily living')
    elif 'A7873972' in dedup_list: # Activity (Level 3) >> Exercise, Walking
        wt = 0.60 # should be regarded as 'func'
        cat = ('desc', 'Activity')
    elif 'A16369886' in dedup_list: # Interpretation of findings (Level 2) >> Diagnosis
        wt = 0.00
        cat = ('desc', 'Interpretation of findings')
    elif 'A3356706' in dedup_list: # Clinical history/examination observable (Level 2) >> ST elevation, LV function
        wt = 0.80 
        cat = ('desc', 'Clinical history/examination observable')
    elif 'A7874071' in dedup_list: # Function (Level 2) >> Walking, Renal function
        wt = 0.60 # should be regarded as 'activity'
        cat = ('desc', 'Function')
    elif aui == 'A7874971': # Process (Level 2)
        wt = 0.00
        cat = ('nil', '=Process')
    elif 'A3591013' in dedup_list: # Observable entity (Level 1)
        wt = 0.40
        cat = ('desc', 'Observable entity')
    
    # BODY STRUCTURE *******************************************************************************************
    
    elif 'A24236918' in dedup_list: # Lesion (Level 4) >> necrosis, infarction
        wt = 0.60
        cat = ('dis', 'Lesion')
    elif 'A3555009' in dedup_list: # Mechanical abnormality (Level 4) >> Acute hemorrhage
        wt = 0.60
        cat = ('dis', 'Mechanical abnormality')
    elif 'A3095626' in dedup_list: # Anatomical structure (Level 3) >> coronary artery
        wt = 0.60
        cat = ('loc', 'Anatomical structure')
    elif 'A3567684' in dedup_list: # Morphologically abnormal structure (Level 3) >> necrosis, diverticulitis
        wt = 0.60
        cat = ('dis', 'Morphologically abnormal structure')
    elif 'A3323363' in dedup_list: # Body structure (Level 1)
        wt = 0.40
        cat = ('loc', 'Body structure')
        
    # SUBSTANCE ************************************************************************************************
    
    elif aui == 'A2878286': # Allograft; 'allocraft (procedure)' is frequently misannotated as material
        wt = 0.80
        cat = ('proc_t', '=Allocraft')
    #elif 'A2873362' in dedup_list: # Enzyme (Level 5) >> Amylase
    #    cat = 'enz'
    elif 'A2873197' in dedup_list: # Proton (Level 5) >> Proton (h)
        wt = 0.00
        cat = ('nil', '=Proton')
    elif 'A7883521' in dedup_list: # Body material (Level 3) >> Blood
        wt = 0.60
        cat = ('loc', 'Body material')
    elif 'A3016529' in dedup_list: # Drinks (Level 3) >> Drinks
        wt = 0.00
        cat = ('nil', 'Drinks')
    elif 'A2881738' in dedup_list: # Foods (Level 3) >> egg fries
        wt = 0.00
        cat = ('nil','Foods')
    elif 'A3161951' in dedup_list: # Natural material (Level 3) >> Tree resins
        wt = 0.00
        cat = ('nil', 'Natural material')
    elif 'A6938913' in dedup_list: # Drug or medicament (level 2)
        wt = 0.80
        cat = ('med', 'Drug or medicament')
    elif 'A16967690' in dedup_list: # Chemical (level 3) >> creatinine, penicillin
        wt = 0.80
        cat = ('sub', 'Chemical')
    elif 'A3738091' in dedup_list: # Substance categorized by physical state (Level 2) >> Liquid
        wt = 0.40
        cat = ('sub', 'Substance categorized by physical state')
    elif 'A3206010' in dedup_list: # Substance (Level 1) >> creatinine
        wt = 0.60
        cat = ('sub', 'Substance')
    
    # PHARMACEUTICAL / BIOLOGIC PRODUCT *************************************************************************
    
    elif 'A2998573' in dedup_list: # Blood derivative product (Level 2) >> Serum product
        wt = 0.60
        cat = ('med', 'Blood derivative product')
    elif 'A6921106' in dedup_list: # Blood product (Level 2) > Human platelets, pooled
        wt = 0.60
        cat = ('med', 'Blood product')
    elif aui == 'A3394388': # Dietary product (Level 2) 
        wt = 0.00
        cat = ('nil', '=Dietary product')
    elif 'A3394388' in dedup_list: # Dietary product (Level 2) >> Infant formula + iron
        wt = 0.60
        cat = ('med', 'Dietary product')
    elif 'A3030066' in dedup_list: # Herbal medicine (Level 2) >> Ginkgo biloba preparation
        wt = 0.60
        cat = ('med', 'Herbal medicine')
    elif aui == 'A29562204': # Medicinal product (Level 2)
        wt = 0.00
        cat = ('nil', '=Medicinal product')
    elif 'A29562204' in dedup_list: # Medicinal product (Level 2) >> Vaccine product containing only Dengue virus antigen 
        wt = 0.80
        cat = ('med', 'Medicinal product')
    elif 'A3627347' in dedup_list: # Pharmaceutical / biologic product (Level 1) >> Pharmaceutical Preparations (medicaion)
        wt = 0.60
        cat = ('med', 'Pharmaceutical / biologic product') 
                
    # CLINICAL FINDING *****************************************************************************************
    # May be an answer to 'observable entity'
    # May equivalent to 'Procedure'

    elif aui == 'A3283608': # Gait normal (Level 8) == normal
        wt = 0.00
        cat = ('nil', '=Gait normal')
    elif aui == 'A15108798': # Macrophage activation syndrome (Level 7) == macrophage activation syndrome
        wt = 0.00
        cat = ('nil', '=Macrophage activation syndrome')
    elif 'A9417638' in dedup_list: # Vision test distance (Level 6) >> 1/3 meter (likely to misannotate'near' to '1/3 meter')
        wt = 0.00
        cat = ('nil', 'Vision test distance')
    elif 'A6917682' in dedup_list: # Complaint (Level 5) == Complaint
        wt = 0.00
        cat = ('nil', '=Complaint')
    elif 'A2957588' in dedup_list: # Chief complaint (Level 5) == Chief complaint
        wt = 0.00
        cat = ('nil', '=Chief complaint') 
    elif 'A3487902' in dedup_list: # Health-related behavior finding (Level 5) >> Tobacco smoking behavior - finding (smoking)
        wt = 0.60
        cat = ('desc', 'Health-related behavior finding')
    elif 'A3323390' in dedup_list: # Body temperature finding (Level 5) >> Fever
        wt = 0.60
        cat = ('dis', 'Body temperature finding')
    elif 'A3558428' in dedup_list: # Mental state finding (Level 5) >> Lightheadedness
        wt = 0.60
        cat = ('dis', 'Mental state finding')
    elif 'A3880798' in dedup_list: # Dizziness (Level 5) >> Vertigo
        wt = 0.60
        cat = ('dis', 'Dizziness')
    elif 'A3709940' in dedup_list: # Social and personal history finding (Level 4) >> Returning home
        wt = 0.00
        cat = ('desc', 'Social and personal history finding')
    elif aui == 'A2873229': # Problem (Level 4)
        wt = 0.00
        cat = ('nil', '=Problem')
    elif 'A2873229' in dedup_list: # Problem (Level 4)
        wt = 0.20
        cat = ('desc', 'Problem')
    elif 'A3709956' in dedup_list: # Social context finding (Level 4) >> Financially poor (poor)
        wt = 0.20
        cat = ('desc', 'Social context finding')
    elif aui == 'A3487977': # Hearing finding (Level 4)
        wt = 0.00
        cat = ('nil', '=Hearing finding')
    elif aui == 'A3532294': # Language finding (Level 4)
        wt = 0.00
        cat = ('nil', '=Language finding')
    elif 'A3323245' in dedup_list: # Body measurement finding (Level 4) >> Weight gain
        wt = 0.60
        cat = ('dis', 'Body measurement finding')
    elif 'A13356695' in dedup_list: # Finding related to biological sex (Level 3) >> Male, Baby female
        wt = 0.00
        cat = ('sex', 'Finding related to biological sex')
    elif aui == 'A3469753': # Functional finding (Level 3)
        wt = 0.00
        cat = ('nil', '=Functional finding')
    elif 'A3473506' in dedup_list: # General finding of observation of patient (Level 3) >> Obese
        wt = 0.40
        cat = ('dis', 'General finding of observation of patient')
    elif aui == 'A2880798': # Disease (Level 2)
        wt = 0.00
        cat = ('nil', '=Disease')
    elif 'A2880798' in dedup_list: # Disease (Level 2) >> Acute myocardial infarction
        wt = 1.00
        cat = ('dis', 'Disease')        
    elif 'A3356704' in dedup_list: # Clinical history and observation findings (Level 2) >> Prodrome, Single lobe lung infiltrate
        wt = 0.40
        cat = ('dis', 'Clinical history and observation findings')
    elif aui == 'A16960111' in dedup_List: # Evaluation finding (Level 2)
        wt = 0.00
        cat = ('nil', '=Evaluation finding')
    elif 'A16960111' in dedup_list: # Evaluation finding (Level 2) >> hypercholesterolemia, HIV Seropositivity
        wt = 0.80
        cat = ('dis', 'Evaluation finding')
    elif 'A3456474' in dedup_list: # Finding by site (Level 2) >> Left upper quadrant pain, Rhinorrhea
        wt = 0.60
        cat = ('dis', 'Finding by site')
    elif 'A3473457' in dedup_list: # General clinical state finding (Level 2) >> Increased pressure
        wt = 0.60
        cat = ('dis', 'General clinical state finding')
    elif aui == 'A3362993': # Color finding (Level 2)
        wt = 0.00
        cat = ('nil', '=Color finding')
    elif 'A3362993' in dedup_list: # Color finding (Level 2) >> Dark color
        wt = 0.20
        cat = ('desc', 'Color finding')
    elif 'A3900203' in dedup_list: # Adverse incident outcome categories (Level 2) >> Adverse incident resulting in death
        wt = 0.20
        cat = ('desc', 'Adverse incident outcome categories')
    elif 'A3242802' in dedup_list: # Administrative statuses (Level 2) >> report status 
        wt = 0.00
        cat = ('desc', 'Administrative statuses')
    elif 'A3886745' in dedup_list: # Clinical finding (Level 1)
        wt = 1.00
        cat = ('dis', 'Clinical finding')
    
    # PROCEDURE *************************************************************************************************
    
    elif aui == 'A3629808': # Physical examination procedure (Level 5)
        wt = 0.00
        cat = ('nil', '=Physical examination procedure')
    elif aui == 'A7881343': # Review of systems (Level 4) == Review of systems
        wt = 0.00
        cat = ('nil', '=Review of systems')
    elif aui == 'A13012866': # Imaging (Level 4) == Imaging
        wt = 0.00
        cat = ('nil', '=Imaging')
    elif 'A3124048' in dedup_list: # Elevation (Level 4) >> Elevation of bone fragments for fracture of orbit
        wt = 0.40
        cat = ('proc_t', 'Elevation')
    elif aui == 'A3300149': # Therapeutic procedure (Level 3) == Therapeutic procedure
        wt = 0.00
        cat = ('nil', '=Therapeutic procedure')
    elif aui == 'A3440255': # Evaluation procedure (Level 3) == evaluation procedure
        wt = 0.00
        cat = ('nil', '=Evaluation procedure')
    elif 'A3440255' in dedup_list: # Evaluation procedure (Level 3) >> White blood cell count
        wt = 0.60
        cat = ('proc_d', 'Evaluation procedure')
    elif 'A3300149' in dedup_list: # Therapeutic procedure (Level 3) >> Therapeutic radiology procedure (radiation therapy)
        wt = 0.80
        cat = ('proc_t', 'Therapeutic procedure')
    elif aui == 'A3074868': # Surgical procedure (Level 3) == Surgical procedure
        wt = 0.00
        cat = ('nil', '=Surgical procedure')
    elif 'A3074868' in dedup_list: # Surgical procedure (Level 3) >> Operative procedure on knee (knee surgery)
        wt = 0.80
        cat = ('proc_t', 'Surgical procedure')
    elif 'A3392578' in dedup_list: # Diagnostic procedure by site >> Diagnostic catetherization of urinary bladder
        wt = 0.80
        cat = ('proc_d', 'Diagnostic procedure by site')
    elif 'A2877542' in dedup_list: # Laboratory procedure (Level 2) >> Hematocrit
        wt = 0.80
        cat = ('proc_d', 'Laboratory procedure')
    elif 'A3647338' in dedup_list: # Procedure by method (Level 2) >> Surgical procedure
        if 'therapeutic' in dedup_token_list:
            wt = 0.80
            cat = ('proc_t', 'therapeutic')
        elif 'evaluation' in dedup_token_list:
            wt = 0.80
            cat = ('proc_d', 'evaluation')
        else:
            wt = 0.60
            cat = ('proc', 'Procedure by method')
    elif 'A3647344' in dedup_list: # Procedure by site (Level 2) >> Cauterization of conjunctiva
        if 'therapeutic' in dedup_token_list:
            wt = 0.80
            cat = ('proc_t', 'therapeutic')
        elif 'evaluation' in dedup_token_list:
            wt = 0.80
            cat = ('proc_d', 'evaluation')
        else:
            wt = 0.60
            cat = ('proc', 'Procedure by site')
    elif 'A16372983' in dedup_list: # Procedure categorized by device involved (Level 2) >> Cardiac catheterization
        if 'therapeutic' in dedup_token_list:
            wt = 0.80
            cat = ('proc_t', 'therapeutic')
        elif 'evaluation' in dedup_token_list:
            wt = 0.80
            cat = ('proc_d', 'evaluation')
        else:
            wt = 0.60
            cat = ('proc', 'Procedure categorized by device')
    elif 'A3092627' in dedup_list: # Administrative procedure (Level 2) >> hospital admission
        wt = 0.00
        cat = ('proc', 'Administrative procedure')
    elif aui == 'A2895444': # == Procedure (Level 1)
        wt = 0.00
        cat = ('nil', '=Procedure')
    elif 'A2895444' in dedup_list: # Procedure (Level 1) >> Echocardiography
        if 'therapeutic' in dedup_token_list:
            wt = 0.80
            cat = ('proc_t', 'therapeutic')
        elif 'evaluation' in dedup_token_list:
            wt = 0.80
            cat = ('proc_d', 'evaluation')
        else:
            wt = 0.60
            cat = ('proc', 'Procedure')
    
    # ORGANISM *************************************************************************************************
    
    elif aui == 'A2882492': # Homo sapiens (Level 12) 
        wt = 0.00
        cat = ('nil', 'Homo sapiens') # this should get wt of 0 to disambiguate the word 'man'
    elif 'A6921673' in dedup_list: # Organism (Level 1) >> bacteria
        wt = 0.60
        cat = ('desc', 'Organism')
        
    # STAGING AND SCALES ***************************************************************************************
    
    elif aui == 'A3161523': # N+ (Level 5)
        wt = 0.00 
        cat = ('nil', '=N+')
    #elif 'A3098696' in dedup_list: # Assessment scales (Level 2) >> pack years
    #    cat = 'ass_scale'
    elif 'A3204387' in dedup_list: # Staging and Scales (Level 1)
        wt = 0.00
        cat = ('nil', 'Staging and scales')
    
    # QUALIFIER VALUE **********************************************************************
    
    # -- Action
    elif aui == 'A3566528': # Monitoring - action (Level 4) == Monitoring
        wt = 0.00
        cat = ('nil', '=Monitoring - action')
    elif 'A3192955' in dedup_list: # Relative times (Level 4) >> before
        wt = 0.00
        cat = ('nil', 'Relative times')
    elif aui == 'A13009256': # Evaluation - action (Level 3)
        wt = 0.00
        cat = ('nil', '=Evaluation - action')
    elif 'A13009256' in dedup_list: # Evaluation - action (Level 3) >> Magnetic resonance imaging - action
        wt = 0.80
        cat = ('proc_d', 'Evaluation - action')
    elif 'A3241349' in dedup_list: # Surgical action (Level 3) >> Dissection - action
        wt = 0.80
        cat = ('proc_t', 'Surgical action')
    elif 'A3591184' in dedup_list: # Obstetric action (Level 3) >> Delivery - action
        wt = 0.80
        cat = ('proc_t', 'Obstetric action')
    elif 'A3241327' in dedup_list: # Removal - action (Level 3) >> Excision debridement - action
        wt = 0.80
        cat = ('proc_t', 'Removal - action')
    elif 'A3214246' in dedup_list: # Types (Level 3) >> Jones
        wt = 0.00
        cat = ('nil', 'Types')
    elif aui == 'A3091961': # Action
        wt = 0.00
        cat = ('nil', '=Action')
    elif 'A3091961' in dedup_list: # Action (Level 2) >> Auscultation - action
        if 'therapeutic' in dedup_token_list:
            wt = 0.80
            cat = ('proc_t', 'therapeutic')
        elif 'evaluation' in dedup_token_list:
            wt = 0.80
            cat = ('proc_d', 'evaluation')
        else:
            wt = 0.60
            cat = ('proc', 'Action')
    
    # -- clinical specialty
    elif 'A10884570' in dedup_list: # Clinical specialty (Level 2) >> cardiology
        wt = 0.20
        cat = ('desc', 'Clinical specialty')
    
    # -- Context value
    elif 'A3632497' in dedup_list: # Planned (Level 6) == Planned
        wt = 1.00
        cat = ('tmp', 'Planned')
    elif 'A8392624' in dedup_list: # Scheduled - procedure estatus
        wt = 1.00
        cat = ('tmp', 'Scheduled - procedure status')

    # -- Finding status values
    elif aui == 'A3264703': # Changed status (Level 4)
        wt = 0.00
        cat = ('nil', '=Changed status')
    elif aui == 'A3108139': # Certainties (Level 3)
        wt = 0.00
        cat = ('nil', '=Certainties')
    elif 'A3281759' in dedup_list: # Finding status values (Level 2) >> Worsening
        wt = 0.20
        cat = ('desc', 'Finding status values')
    
    # -- Disposition
    elif 'A28418399' in dedup_list: # Disposition (Level 2) >> Antiviral
        wt = 0.60 # ='Substance'
        cat = ('sub', 'Disposition')
    
    # -- Dose form intended site
    elif 'A28896694' in dedup_list: # Dose form intended site (Level 2) >> Enteral
        wt = 0.20 # = 'Spatial and relational concepts', 'Route of administration value'
        cat = ('desc', 'Dose from intended site')
        
    # -- Findings values
    elif aui == 'A3060341': # Present (Level 4)
        wt = 0.00
        cat = ('nil', '=Present')
    elif 'A3089702' in dedup_list: # Absence findings (Level 3) >> Negative, Lacking
        wt = 0.00
        cat = ('nil', '=Absence findings')
    elif 'A3132009' in dedup_list: # Findings values (Level 2) >> Increasing, Positive
        wt = 0.20
        cat = ('desc', 'Findings values')
        
    # -- General clinical stage for disease AND/OR neoplasm
    elif 'A3473454' in dedup_list: # General clinical stage for disease AND/OR neoplasm (Level 2) >> Late stage
        wt = 0.20
        cat = ('desc', 'General clinical stage for disease AND/OR neoplasm')
        
    # -- Mechanism of disease spread value
    elif 'A23454121' in dedup_list: # Mechanism of disease spread value (Level 2) >> Hematogenous spread
        wt = 0.20
        cat = ('desc', 'Mechanism of disease spread value')
        
    # -- Mechanisms
    elif aui == 'A2887024': # Surgery (Level 3):
        wt = 0.00
        cat = ('nil', '=Surgery')
    elif aui == 'A3157401': # Mechanisms (Level 2)
        wt = 0.00
        cat = ('nil', '=Mechanisms')
    elif 'A3157401' in dedup_list: # Mechanisms (Level 2) >> Barotrauma mechanisms
        wt = 0.20
        cat = ('desc', 'Mechanisms')
    
    # -- Modifiers of Analytes and Substances
    elif 'A3566118' in dedup_list: # Modifiers of Analytes and Substances (Level 2) >> Alpha subunit
        wt = 0.20
        cat = ('desc', 'Modifiers of analytes and substances')
        
    # -- Pharmaceutical dose form
    elif 'A28878916' in dedup_list: # Pharmaceutical dose form (Level 2) >> Conventional release oral syrup (oral syrup)
        wt = 0.20
        cat = ('desc', 'Pharmaceutical dose form')
        
    # -- Precondition value
    elif 'A24090101' in dedup_list: # Precondition value (Level 2) >> At rest
        wt = 0.20
        cat = ('desc', 'Precondition value')
    
    # -- Process
    elif aui == 'A27784444': # Process (Level 2)
        wt = 0.00
        cat = ('nil', '=Process')
    elif 'A27784444' in dedup_list: # Process (Level 2) >> Psychogenic, Inflammation
        wt = 0.40
        cat = ('desc', 'Process')
        
    # -- Ranked categories
    elif aui == 'A3137011': # Grades (Level 4)
        wt = 0.00
        cat = ('nil', '=Grades')
    elif 'A3137011' in dedup_list: # Grades (Level 4) >> Mild
        wt = 0.40
        cat = ('desc', 'Grades')
    elif aui == 'A3110547': # Classes (Level 3)
        wt = 0.00
        cat = ('nil', '=Classes')
    elif aui == 'A3123635': # Editions (Level 3)
        wt = 0.00
        cat = ('nil', '=Editions')
    elif aui == 'A3137422': # Groups (Level 3)
        wt = 0.00
        cat = ('nil', '=Groups')
    elif 'A3137422' in dedup_list: # Groups (Level 3) >> Group A
        wt = 0.60
        cat = ('desc', 'Groups')
    elif aui == 'A3152689': # Levels (Level 3)
        wt = 0.00
        cat = ('nil', '=Levels')
    elif 'A3152689' in dedup_list: # Levels (Level 3) >> Intermediate
        wt = 0.60
        cat = ('desc', 'Levels')
    elif aui == 'A3198151': # Scores (Level 3)
        wt = 0.00
        cat = ('nil', '=Scores')
    elif aui == 'A3204385': # Stages (Level 3)
        wt = 0.00
        cat = ('nil', '=Stages')
    elif 'A3204385' in dedup_list: # Stages (Level 3) >> Stage 1A
        wt = 0.60
        cat = ('desc', 'Stages')
    elif aui == 'A3214246': # Types (Level 3)
        wt = 0.00
        cat = ('nil', '=Types')
    elif 'A3214246' in dedup_list: # Types (Level 3) >> Serotype H3N2
        wt = 0.60
        cat = ('desc', 'Types')
    elif 'A3191245' in dedup_list: # Ranked categories (Level 2)
        wt = 0.20
        cat = ('desc', 'Ranked categories')
        
    # -- Result comments
    elif 'A3194939' in dedup_list: # Result comments (Level 2) >> Normal result
        wt = 0.20
        cat = ('desc', 'Result comments')
        
    # -- Route of administration value
    elif 'A6947385' in dedup_list: # Route of administration value (Level 2) >> Intravenous route
        wt = 0.20 # = 'Dose form intended site', 'Spatial and relational concepts'
        cat = ('desc', 'Route of administration value')
    
    # -- Spatial and relational concepts
    elif aui == 'A3200743': # Side (Level 4)
        wt = 0.00
        cat = ('nil', '=Side')
    elif 'A3203040' in dedup_list: # Specific site descriptor (Level 4) >> Mesenteric
        wt = 0.40
        cat = ('loc', 'Specific site descriptor')
    elif 'A3200743' in dedup_list: # Side (Level 4) >> Left
        wt = 1.00
        cat = ('side', 'Side')
    elif 'A3200754' in dedup_list: # Sidedness >> Left sided
        wt = 1.00
        cat = ('side', 'Sidedness')
    elif 'A3268221' in dedup_list: # Spatial and relational concepts (Level 2) >> 
        wt = 0.20 # = 'Dose form intended site', 'Route of administration value'
        cat = ('desc', 'Spatial and relational concepts')
        
    # -- Sport
    elif 'A7873282' in dedup_list: # Sport >> Baseball
        wt = 0.20
        cat = ('desc', 'Sport')
    
    # -- State of matter
    elif 'A28898878' in dedup_list: # State of matter >> Liquid
        wt = 0.20
        cat = ('desc', 'State of matter')
        
    # -- Technique
    elif aui == 'A24089189': # Technique (Level 2)
        wt = 0.00
        cat = ('nil', '=Technique')
    elif 'A24089189' in dedup_list: # Technique (Level 2) >> Imaging technique, Mechanical
        wt = 0.40
        cat = ('desc', 'Technique')
        
    # -- Time frame
    elif 'A3755612' in dedup_list: # Time frame (Level 2) >> Daily, Periodic
        wt = 0.20
        cat = ('desc', 'Time frame')

    # -- Type of drug preparation
    elif 'A3393038' in dedup_list: # Dialysis dosage form (Level 3) >> Hemodialysis solution
        wt = 0.20
        cat = ('desc', 'Dialysis dosage form')
    elif 'A3657907' in dedup_list: # Radiopharmaceutical dosage form (Level 3) >> Radioactive implant
        wt = 0.20
        cat = ('desc', 'Radiopharmaceutical dosage form')
    elif 'A3768554' in dedup_list: # Type of drug preparation (Level 2) >> Inhaler
        wt = 0.40 # = 'Unit of presentation', 'Device'
        cat = ('sub', 'Type of drug preparation')
    
    # -- Unit of measure
    elif 'A7880590' in dedup_list: # Percentage unit (Level 4) >> % positive cells
        wt = 0.00
        cat = ('nil', 'Percentage unit')
    elif aui == 'A3087908': # Per forty (Level 4) == /40
        wt = 0.00
        cat = ('nil', 'Per forty')
    elif 'A29521976' in dedup_list: # Unit of measure (Level 2) >> Yards, Months
        wt = 0.20 
        cat = ('desc', 'Unit of measure')
    
    elif 'A3030642' in dedup_list: # History of (Level 3) == History of
        wt = 0.40
        cat = ('desc', 'History of')

    # -- Descriptor
    elif 'A3243208' in dedup_list: # Times relative to admission (Level 6) >> On admission
        wt = 0.00
        cat = ('desc', 'Times relative to admission')
    elif 'A3297078' in dedup_list: # Pregnancy time period (Level 6) >> Third trimester
        wt = 0.60
        cat = ('desc', 'Pregnancy time period')
    elif 'A3126026' in dedup_list: # Event orders (Level 5) >> Initially
        wt = 0.00
        cat = ('desc', 'Event orders')
    elif 'A3134454' in dedup_list: # Frequencies (Level 4) >> Continuous, Daily
        wt = 0.20
        cat = ('desc', 'Frequencies')
    elif 'A3100796' in dedup_list: # Behavior descriptors (Level 3) >> complicated, Hemolytic
        wt = 0.40
        cat = ('desc', 'Behavior descriptors')
    elif aui == 'A3318667': # Flaccidity (Level 3) == Flaccidity
        wt = 0.40
        cat = ('desc', 'Flaccidity')
    elif 'A3133890' in dedup_list: # Formations (Level 3) >> Bifurcated
        wt = 0.40
        cat = ('desc', 'Formations')
    elif aui == 'A2933364': # Hyperkinetic (Level 3) == Hyperkinetic
        wt = 0.40
        cat = ('desc', 'Hyperkinetic')
    elif 'A3143175' in dedup_list: # Hypermobility (Level 3) >> Hypermobile
        wt = 0.40
        cat = ('desc', 'Hypermobility')
    elif aui == 'A3143198': # Hyperreflexic (Level 3) == Hyperreflexic
        wt = 0.40
        cat = ('desc', 'Hyperreflexic')
    elif aui == 'A2933441': # Hypokinetic (Level 3) == Hypokinetic
        wt = 0.40
        cat = ('desc', 'Hypokinetic')
    elif 'A3159860' in dedup_list: # Morphological descriptors (Level 3) >> Acinar
        wt = 0.40
        cat = ('desc', 'Morphological descriptors')
    elif aui == 'A3296880': # Pathogeneses 
        wt = 0.00
        cat = ('nil', '=Pathogeneses')
    elif 'A3296880' in dedup_list: # Pathogeneses (Level 3) >> Drug-induced
        wt = 0.40
        cat = ('desc', 'Pathogeneses')
    elif 'A2921405' in dedup_list: # Psychologic (Level 3) >> Psychologic
        wt = 0.40
        cat = ('desc', 'Psychologic')
    elif 'A3189864' in dedup_list: # Pulsatile (Level 3) == Pulsatile
        wt = 0.40
        cat = ('desc', 'Pulsatile')
    elif aui == 'A3192915': # Relationships
        wt = 0.00
        cat = ('nil', '=Relationships')
    elif 'A3192915' in dedup_list: # Relationships (Level 3) >> Autologous
        wt = 0.20
        cat = ('desc', 'Relationships')
    elif 'A6947587' in dedup_list: # Sensation quality (Level 3) >> Burning sensation quality (burning)
        wt = 0.20
        cat = ('desc', 'Sensation quality')
    elif 'A3199217' in dedup_list: # Sensibilities (Level 3) >> Olfactory
        wt = 0.40
        cat = ('desc', 'Sensibilities')
    elif aui == 'A2946316': # Spasticity (Level 3) == Spasticity
        wt = 0.60 # = Clinical Finding >> Finding by site
        cat = ('dis', 'Spasticity')
    elif 'A3120607' in dedup_list: # Directions (Level 3) >> Elevation - value
        wt = 0.20
        cat = ('desc', 'Directions')
    elif aui == 'A3269797': # Courses
        wt = 0.00
        cat = ('nil', '=Courses')
    elif aui == 'A3121378': # Distributions
        wt = 0.00
        cat = ('nil', '=Distributions')
    elif aui == 'A3122687': # Durations
        wt = 0.00
        cat = ('nil', '=Durations')
    elif aui == 'A3134454': # Frequencies
        wt = 0.00
        cat = ('nil', '=Frequencies')
    elif aui == 'A3171632': # Occurrences
        wt = 0.00
        cat = ('nil', '=Occurrences')
    elif aui == 'A3188236': # Priorities
        wt = 0.00
        cat = ('nil', '=Priorities')
    #elif 'A3192955' in dedup_list: # Relative times (Level 4) >> After values (post)
    #    wt = 1.00
    #    cat = ('desc', 'Relative times')
    elif 'A8392622' in dedup_list: #  = Scheduled - priority (Level 5) 
        wt = 1.00
        cat = ('tmp', 'Scheduled - priority')
    elif 'A3210757' in dedup_list: # Time patterns (Level 3) >> Acquired, Preoperative, Secondary
        wt = 0.40
        cat = ('desc', 'Time patterns')
    elif 'A3174565' in dedup_list: # Ordinal number (Level 3) >> Secondary
        wt = 0.40
        cat = ('desc', 'Ordinal number')
    elif 'A3214887' in dedup_list: # Uniformities (Level 3) >> Irregular
        wt = 0.20
        cat = ('desc', 'Uniformities')
    elif aui == 'A3217541': # Velocities
        wt = 0.00
        cat = ('nil', '=Velocities')
    elif 'A7873501' in dedup_list: # Descriptor (Level 2) >> Slow, More
        wt = 0.00
        cat = ('desc', 'Descriptor')
    
    # -- Dosing instruction fragment
    elif 'A10885554' in dedup_list: # Dosing instruction fragment (Level 2) >> During - dosing instruction fragment
        wt = 0.0
        cat = ('nil', 'Dosing instruction fragment')
    
    elif 'A3655404' in dedup_list: # Qualifier (Level 1)
        wt = 0.00
        cat = ('desc', 'Qualifier')
    
    # SOCIAL CONTEXT *****************************************************************************************
    
    #elif 'A2874675' in dedup_list: # Man (Level 5) == Men
    #    cat = 'male' # also in 'Clinical finding' category
    #elif 'A2874673' in dedup_list: # Woman (Level 5) == Women
    #    cat = 'female' # also in "Clinical finding" category
    
    elif 'A3285870' in dedup_list: # Healthcare professional (Level 4) >> doctor
        wt = 0.00
        cat = ('desc', 'Healthcare professional')
    elif 'A13021175' in dedup_list: # Person in the healthcare environment (Level 3) >> patient
        wt = 0.00
        cat = ('desc', 'Person in the healthcare environment')
    elif 'A6945981' in dedup_list: # Person categorized by age (Level 3) >> Woman
        wt = 0.00
        cat = ('sex', 'Person categorized by age')
    elif 'A7881113' in dedup_list: # Racial group (Level 3) >> Caucasian (white)
        wt = 0.00
        cat = ('desc', 'Racial group')
    elif 'A3183829' in dedup_list: # Person (Level 2)
        wt = 0.00
        cat = ('desc', 'Person')
    
    # PHYSICAL OBJECT ******************************************************************************************
    
    elif 'A3256591' in dedup_list: # Biomedical device (Level 4) >> Prosthesis
        wt = 0.80
        cat = ('sub', 'Biomedical device')
    elif 'A10884564' in dedup_list: # Clinical equipment and/or device (Level 3) >> Audiometric room
        wt = 0.40
        cat = ('sub', 'Clinical equipment and/or device')
    elif 'A3629886' in dedup_list: # Physical object (Level 1) >> Armband
        wt = 0.20
        cat = ('sub', 'Physical object')
        
    # ENVIRONMENT OR GEOGRAPHICAL LOCATION *********************************************************************
    
    elif aui == 'A3299532': # == Study
        wt = 0.00
        cat = ('nil', '=Study')
    elif 'A2981767' in dedup_list: # St. Lucia (Level 5) == St. Lucia
        wt = 0.00 # Often misannotated from 'st'
        cat = ('nil', 'St. Lucia')
    elif 'A3286724' in dedup_list: # Location within hospital premises (Level 4) >> Intensive care unit
        wt = 0.20
        cat = ('desc', 'Location within hospital premises')
    elif 'A3819332' in dedup_list: # Site of care (Level 3) >> medical center
        wt = 0.20
        cat = ('desc', 'Site of care')
    elif 'A10886351' in dedup_list: # Environment or geographical location (Level 1) >> Medical intensive care unit
        wt = 0.40
        cat = ('desc', 'Environment or geographical location')
    
    # Event ****************************************************************************************************
    elif 'A9405404' in dedup_list: # Accidental event (Level 3) >> accidental fall
        wt = 0.6
        cat = ('dis', 'Accidental event')
        
    # SPECIAL CONCEPT ******************************************************************************************
    elif 'A3575676' in dedup_list: # Navigational concept (Level 2) >> Soft tissue
        wt = 0.40
        cat = ('loc', 'Navigational concept')
        
    else:
        wt = 0.00
        cat = ('nil', 'ns')
    
    # SNOMED CT Model Component 
    
    # elif 'A6919920' in dedup_list: # Date (Level 6) == date
    #    cat = 'date'
    #elif 'A21520839' in dedup_list: # SNOMED CT Model Component (Level 1)
    #    wt = 0.00
    #    cat = ('nil', 'SNOMED CT model component')
    
    return cat, wt

In [13]:
def from_str_to_aui(string):
    sql = 'select distinct aui from mrconso where str = %s and tty = %s'
    val = (string, 'pt')
    mycursor.execute(sql, val)
    result = mycursor.fetchall()
    if result:
        return [i[0] for i in result]

In [17]:
def bigrams(string): # returns a list of possible bigrams
    bigramList = []
    stringList = string.split(' ')
    c = 0
    while c < len(stringList) - 1:
        bigramList.append(stringList[c]+' '+stringList[c+1])
        c += 1
    return bigramList

In [18]:
def trigrams(string): # returns a list of possible trigrams
    trigramList = []
    stringList = string.split(' ')
    c = 0
    while c < len(stringList) - 2:
        trigramList.append(stringList[c]+' '+stringList[c+1]+' '+stringList[c+2])
        c += 1
    return trigramList

In [19]:
def range_subset(range1, range2):
    """Whether range1 is a subset of range2."""
    if not range1:
        return True  # empty range is subset of anything
    if not range2:
        return False  # non-empty range can't be subset of empty range
    if len(range1) > 1 and range1.step % range2.step:
        return False  # must have a single value or integer multiple step
    return range1.start in range2 and range1[-1] in range2

# Replace document with AUI and PTs

In [20]:
@timeout_decorator.timeout(90) 
def replace_aui(psent): # v.5.0 used SNOMED CT Snowstorm API
    # input is a preprocessed string sentence (not a list!!!)
    """
    Input: ' The patient underwent a MRI / MRA of the head , neck and upper chest which demonstrated a proximal 
    left subclavian artery stenosis , proximal to the take off of the left vertebral artery .'
    
    Output: ' patient underwent a MAGNETIC_RESONANCE_IMAGING / MAGNETIC_RESONANCE_ANGIOGRAPHY ANGIOGRAPHY HEAD , 
    NECK and UPPER_CHEST which demonstrated a proximal LEFT_SUBCLAVIAN_ARTERY SUBCLAVIAN_ARTERY_STENOSIS STENOSIS ,
    proximal to take off LEFT_VERTEBRAL_ARTERY .'
    """
    sw = stopwords.words('english')
    headersList = ['chief complaint', 'c / c', 'cc', 'reason for visit',
               'history of present illness', 'hpi', 'history of the present illness', 
                   'history of presenting complaint', 'history of the presenting complaint', 
                   'history of present complaint', 'history of the present complaint',
                   'interval history',
               'medical history', 'past medical history', 'surgical history', 'past medical / surgical history', 
                   'past surgical history', 
               'family history', 'fh', 'fhx', 
                'social history', 'shx', 
               'review of system', 'ros', 
               'physical examination', 'physical exam', 'exam',
               'allergy', 'allergies', 'adverse reaction',
               'medication', 'current medication', 'current med', 'home med', 'home medication', 
                   'medications at home', 'medication at home', 'meds at home', 'med at home', 
                   'medications on admission', 'medication on admission', 'meds on admission', 'med on admittion', 'active medication', '\nmed', 
               'lab', 'laboratory', 'laboratories', 'laboratory data', 'lab evaluation', 'laboratory evaluation', 
                   'data',
               'radiological study', 'radiological studies', 'radiology', 'radiological exam', 
                   'radiological examination', 'diagnostic studies', 'studies', 'radiology of chest', 
                   'computed tomography', 'chest computed tomography', 'abdomen computed tomography', 
                   'magnetic resonance imaging', 'echo', 'electrocardiogram',
               'problem', 'problem list', 
               'impression', 'assessment', 'assessment and plan', 'assessment / plan', 'assessment & plan', 
                   'a / p',
               'plan', 'recommendation', 
                   'assessment and plan', 'assessment / plan', 'assessment & plan']
    topLevelConcepts = ['A3323363', 'A3886745', 'A10886351', 'A9379877', 'A3591013', 'A6921673', 'A3627347', 
                        'A3057938', 'A3629886', 'A2895444', 'A3655404', 'A9415425', 'A10894596', 'A21483325', 
                        'A21520839', 'A3818834', 'A3713095', 'A3714377', 'A3204387', 'A3206010', 'A3206011']
    #rsent = " "
    #psent = psent.replace(' of ', ' ') # to match orig_term and expressions in the sentence
    #psent = psent.replace(' the ', ' ') # to match orig_term and expressions in the sentence
    #psent = psent.replace(' for ', ' ')
    psent = psent.replace(' / ', ' ')
    # Delete subheadings
    psent = psent.replace('history of ', '')
    psent = psent.replace('status post ', '')
    psent = psent.replace('head eyes ears nose throat ', ' ')
    psent = psent.replace('cardiac : ', ' ')
    psent = psent.replace('pulmonary : ', ' ')
    psent = psent.replace('abdoment : ', ' ')
    psent = psent.replace('extr : ', ' ')
    psent = psent.replace('extremities : ', ' ')
    psent = psent.replace('neurologic : ', ' ')
    # Erase 'back' if it is adverb to a verb, not a anatomical region.
    for backWord in ['came', 'come', 'comes', 'give', 'gave', 'gives', 'bring', 'brings', 'brought', 
                     'result', 'results', 'go', 'goes', 'went', 'turn', 'turns', 'turned']:
        psent = re.sub(rf'{backWord} (back) ', rf'{backWord} ', psent)

    
    extractedConcepts = extract_raw_concepts(psent)
    # Words with longer characters should be annotated earlier than those with shorter characters.
    # Otherwise, it will annotate 'human immunodeficiency virus' as 'hu{man} immunodeficiency virus', instead of '{HIV}'
    
    cDic = dict()

    for concept in extractedConcepts:
        cui = concept[0]
        triggerInBracket = concept[1].lstrip('[').rstrip(']')
        trigger = re.findall('".+"-tx-\d-"(.+)"-[a-z_]+-\d', triggerInBracket)[0]
        
        #print(cui, trigger)
        pt = concept[2]
        if not (trigger=='adenosine' or trigger=='adult' or trigger=='adults' or trigger=='alcohol' or trigger=='an'  
                or trigger=='apr' or trigger=='attack' or trigger=='attacks' or trigger=='aug'
                or trigger=='bands' or trigger=='beer' or trigger=='blood pressure'
                or trigger=='call' or trigger=='cardiac' or 'catch' in trigger or trigger=='caught' 
                or trigger=='cc' or trigger=='check' or trigger=='checks' or trigger=='cigar' 
                or trigger=='complaint' 
                or trigger=='consult' or trigger=='consultation' or trigger=='consulted' 
                or trigger=='cut' or trigger=='cuts' or trigger=='cutting'
                or 'day' in trigger 
                or trigger=='date' or trigger=='dec' or trigger=='deny' or trigger=='denies' or trigger=='denied'
                or trigger=='dis' or trigger=='discharge' or trigger == 'discuss' or trigger=='discussed' 
                or trigger=='drugs' 
                or trigger=='e 1' or trigger=='e test'
                or trigger=='ethanol' or trigger=='etoh' or trigger=='evaluation' 
                or 'exam' in trigger or trigger=='excess'
                or trigger=='feb' or 'feel' in trigger or trigger=='felt'
                or 'follow up' in trigger or trigger=='female' or trigger=='function' or trigger=='future'
                or trigger=='get' or trigger=='gm' or trigger=='got' or trigger=='gotten'
                or trigger=='hit' or trigger=='history' or trigger=='h 1'
                or trigger=='i' or trigger=='id' or trigger=='idea' or trigger=='ideas' or trigger=='in' 
                or trigger== 'is' 
                or trigger=='jan' or trigger=='jul' or trigger=='jun'
                or trigger=='1' or trigger=='large' or trigger=='left' or trigger=='lesion' or trigger=='lesions' 
                or trigger=='loss'
                or trigger=='male' or trigger=='management' or trigger=='mar' or trigger=='MAS' or trigger=='may' 
                or 'medication' in trigger or 'monitor' in trigger or 'month' in trigger
                or trigger=='mg' or trigger=='near' or trigger=='nov'
                or trigger=='oct' or trigger=='of' or trigger=='old' or trigger=='os' 
                or trigger=='pack' or trigger=='packs' or trigger=='panel' or trigger=='panels' 
                or trigger=='plan' or trigger=='point' or trigger=='prescription' 
                or trigger=='pulmonary'
                or trigger=='remote' or trigger=='respiratory rate' or 'review' in trigger or trigger=='right' 
                or trigger=='risk factors'
                or trigger=='saw' or trigger=='scheduled' or trigger=='sch' or 'show' in trigger   
                or trigger=='see' or trigger=='sep' or trigger=='sex' or trigger=='small'
                or trigger=='smoked' or trigger=='smoker' or trigger == 'smokes' or trigger=='smoking' or 'smok' in trigger
                or trigger=='status post' 
                or trigger=='stress' 
                or trigger=='take' or trigger=='takes' or trigger=='taking' or trigger=='temperature' 
                or trigger=='test' or trigger=='tests' or trigger=='testing' or trigger =='to' 
                or trigger=='tobacco' or trigger=='took' or trigger=='transfer' or trigger=='transferred' 
                or trigger=='treatment'
                or trigger=='underlying' or trigger=='use' or trigger=='used' or trigger=='uses'
                or 'week' in trigger or trigger=='wine' or 'work' in trigger
                or 'year' in trigger 
                or trigger=='XXX' 
                or re.match('[\d./]{1,5}',trigger)): 
            if trigger != sw+headersList:
                #startPos = psent.find(trigger)
                #endPos = startPos + len(trigger)
                #pos = range(startPos, endPos)
                auiList = []
                if to_aui(cui):
                    for a in to_aui(cui):
                        if determine_wt(a[0])[1] >= 0.6:
                            if a[0] not in topLevelConcepts:
                                auiList.append(a[0])
                else:
                    if len(trigger) >= 4:
                        try:
                            url = 'https://snowstorm.ihtsdotools.org/snowstorm/snomed-ct/browser/MAIN%2F2016-07-31/descriptions?term=' + trigger + '&active=true&groupByConcept=true&searchMode=STANDARD&offset=0&limit=1'
                            response = requests.get(url)
                            try:
                                fsn = response.json()['items'][0]['concept']['fsn']['term']
                                for term, tag in re.findall('([A-Za-z0-9 ]+) \(([A-Za-z0-9 ]+)\)', fsn):
                                    if ((tag == 'body structure') or (tag == 'disorder') or (tag == 'finding') or 
                                        (tag == 'procedure') or (tag == 'substance')):
                                        faui = from_fsn_to_aui(term)
                                        if faui != None:
                                            if determine_wt(faui)[1] >= 0.6:
                                                if faui not in topLevelConcepts:
                                                    auiList.append(faui)
                                                #print('This is from url:', trigger, '->', term)
                            except:
                                continue
                        except:
                            continue

                auiList = list(set(auiList)) 
                #print('auiList:', auiList, '\n') # auiList = [('A2949132',), ('A3318032',)]
                for au in auiList:
                    cDic.setdefault(trigger,[]).append(au)
    '''
    for k,v in cDic.items():
        for aui in v:
            print(aui, to_str(aui), determine_wt(aui))
    '''
    for rng in cDic:
        cDic[rng] = list(set(cDic[rng]))
    cDic = dict(sorted(cDic.items(), key=lambda x: len(x[0]), reverse = True))
    #print('cDic:', cDic, '\n')
    num = 1
    for word,aui in cDic.items():
        aui = re.sub('\[', '',str(aui))
        aui = re.sub('\]', '',aui)
        aui = re.sub("'", '', aui)
        aui = re.sub(", ", ' ', aui)
        psent = psent.replace(word.lower(), str(aui))
        if len(word.split()) >= 2:
            firstWord = word.split()[0] 
            lastWord = word.split()[-1]
            # the following code is from https://stackoverflow.com/questions/2148700/how-do-i-find-the-shortest-overlapping-match-using-regular-expressions
            # to match the shortest matching phrases
            rg = re.compile(rf"(?=({word.split()[0]}.+?{word.split()[-1]}))", re.DOTALL|re.IGNORECASE)
            matches = rg.findall(psent)
            for mm in matches:
                mm = min(matches, key=len)
                psent = re.sub(rf"{mm}", rf"{aui}", psent)
            #for mm in re.findall(rf"{word.split()[0]}.+?{word.split()[-1]}", psent):
            #    psent = re.sub(rf"{mm}", rf"{cDic[word]}", psent)
        else:
            continue
        #print('iteration', str(num)+':', 'with', word, aui, '\n', psent)
        num += 1

    return psent
    

In [22]:
def replace_cat(rasent): # v.3.1 
    # input is a sentence in which concepts were transformed to AUIs (replace_aui)
    for aui in re.findall('A\d{7,8}', rasent):
        rasent = re.sub(aui, determine_wt(aui)[0][0].upper(), rasent)
    
    return rasent

In [23]:
def replace_pt(rasent): # v.3.0
    # input is a sentence in which concepts were transformed to AUIs (replace_aui)
    for aui in re.findall('A\d{7,8}', rasent):
        rasent = re.sub(aui, to_str(aui).upper(), rasent)
    return rasent

In [26]:
def negate(doc): # v.1.0
    # doc is a replace_aui sentence
    # --- remove negated information
    negated = []
    # no
    for item in re.findall('[^\n\\.:;,]+:\s*none', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]+:\s*no [^\n\\.]*', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    for item in re.findall('[^\n\\.:;,]+:\s+denie[sd]?[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]+:\s+unremarkable[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
            
    # rule out
    for item in re.findall('[^\n\\.:;]*rule[sd]? out[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;]*rule[sd]? \w+ out[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # deny
    for item in re.findall('[^\n\\.:;,]*denie[sd]? [^\n\\.]*', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]*deny.{0,3} [^\n\\.]*', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # cannot see
    for item in re.findall('[^\n\\.:;]*cannot see[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # unlikely
    for item in re.findall('[^\n\\.:;]*unlikely [^\n\\.]*', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    
    # negative for
    for item in re.findall('[^\n\\.:;]*negative for [^\n\\.]+', doc): ### modified by me
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]+:\s*negative', doc): ### modified by me
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
        
    # is negative ### added by me
    for item in re.findall('[^\n\\.;;]*was negative ', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.;;]*were negative ', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.;;]*is negative ', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.;;]*are negative ', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())  
    
    # hold ### added by me
    for item in re.findall('\shold [^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            if (determine_wt(aui)[0][0]=='med') or (determine_wt(aui)[0][0]=='sub') or (determine_wt(aui)[0][0]=='proc_t') or (determine_wt(aui)[0][0]=='proc_d') or (determine_wt(aui)[0][0]=='proc'):
                negated.append(aui.strip())
    for item in re.findall('\sholds [^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            if (determine_wt(aui)[0][0]=='med') or (determine_wt(aui)[0][0]=='sub') or (determine_wt(aui)[0][0]=='proc_t') or (determine_wt(aui)[0][0]=='proc_d') or (determine_wt(aui)[0][0]=='proc'):
                negated.append(aui.strip())
    for item in re.findall('\sholded [^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            if (determine_wt(aui)[0][0]=='med') or (determine_wt(aui)[0][0]=='sub') or (determine_wt(aui)[0][0]=='proc_t') or (determine_wt(aui)[0][0]=='proc_d') or (determine_wt(aui)[0][0]=='proc'):
                negated.append(aui.strip())
    for item in re.findall('\sholding [^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            if (determine_wt(aui)[0][0]=='med') or (determine_wt(aui)[0][0]=='sub') or (determine_wt(aui)[0][0]=='proc_t') or (determine_wt(aui)[0][0]=='proc_d') or (determine_wt(aui)[0][0]=='proc'):
                negated.append(aui.strip())

    # neither
    for item in re.findall('[^\n\\.:;,]*neither [^\n\\.,]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    # nor
    for item in re.findall('[^\n\\.:;,]* nor [^\n\\.,]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not appear
    for item in re.findall('[^\n\\.:;,]*not appear[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not known
    for item in re.findall('[^\n\\.:;,]*not known to[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not appreciate
    for item in re.findall('[^\n\\.:;,]*not appreciate[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not complain
    for item in re.findall('[^\n\\.:;,]*not complain[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not demonstrate
    for item in re.findall('[^\n\\.:;,]+not demonstrate[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not exhibit
    for item in re.findall('[^\n\\.:;,]*not exhibit[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not feel
    for item in re.findall('[^\n\\.:;,]*not feel[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]*not felt [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not reviewed
    for item in re.findall('[^\n\\.:;,]*not reviewed[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # not have
    for item in re.findall('[^\n\\.:;,]*not have [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]*not had [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[^\n\\.:;,]*never had [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # does not
    for item in re.findall('does not [^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('did not see[^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('did not show[^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('did not reveal[^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('did not experience[^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('did not take[^\n\\.]{0,50}', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
            
    # no
    for item in re.findall('\sno [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # without
    doc = re.sub(' w\/o ', ' without ', doc)
    for item in re.findall('without any[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('without evidence[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('without indication[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('without sign[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('without[^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())

    # free of
    for item in re.findall('free of [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('absence of [^\n\\.]+', doc): 
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
            
    # prevention
    for item in re.findall('[^\n\\.]*prevent[^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('A3187543 [^\n\\.]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
        
    # planned, scheduled, recommended
    for item in re.findall('[A\d ]+[a-z ]{0,10}plann?e?d?', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('plann?e?d?[a-z ]{0,10}[A\d ]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[A\d ]+[a-z ]{0,10}scheduled?', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('scheduled?[a-z ]{0,10}[A\d ]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('[A\d ]+[a-z ]{0,10}recommende?d?', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('recommende?d?[a-z ]{0,10}[A\d ]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    
    # possible
    for item in re.findall('[A\d ]+[a-z ]{0,10}possib', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    for item in re.findall('possib[a-z ]{0,15}[A\d ]+', doc):
        for aui in re.findall('A\d{7,8}', item):
            negated.append(aui.strip())
    
        
    return negated

In [30]:
#def proc_file(file):
#f = open("/Users/silve/OneDrive/Python/UMLS/n2c2-2018-t1/train/"+file, "r")
#record = f.read()
#f.close()

def make_preproc_dict(record): # v.2.1
    record = preproc(record)
    record += 'This is the end of patient record .'
    
    # devide doc by visit
    datesList = []
    for recordDate in re.findall('record date : (\d\d\d\d \d\d \d\d)', record):
        datesList.append(recordDate)
    currentDate = datesList[-1]
    cYear = int(currentDate[:4])
    cMonth = int(currentDate[5:7])
    cDay = int(currentDate[8:10])
    
    preproc_dict = dict()

    # build a dictionary (preproc_dict) whose keys are number of days from current date and their values are clinical notes on the corresponding day
    i = 0
    while i < len(datesList):
        date = datesList[i]
        rYear = int(date[:4])
        rMonth = int(date[5:7])
        rDay = int(date[8:10])
        rDate = datetime(year = rYear, month = rMonth, day = rDay)
        cDate = datetime(year = cYear, month = cMonth, day = cDay)
        dDate = cDate - rDate
        dDays = re.search('\d+', str(dDate)).group(0)
        dDays = int(dDays)
        if i < len(datesList)-1:
            startD = 'record date : '+datesList[i]
            endD = 'record date : '+datesList[i+1]
            doc = find_between(record, startD, endD)
        else:
            startD = 'record date : '+datesList[i]
            endD = 'This is the end of patient record .'
            doc = find_between(record, startD, endD)

        preproc_dict[dDays] = {}
        preproc_dict[dDays]['rtext'] = 'chief complaint999 :\n' + doc + '\nThis is the end of the day'
        #preproc_dict[dDays]['rtext'] = doc + '\nThis is the end of the day'
        
        i += 1

    # devide daily record by sections
     
    ccHeaders = ['chief complaint', 'c / c', 'cc', 'reasons? for visit', 'presenting complaint']
    hpiHeaders = ['history of present illness', 'history of the present illness', 'hpi', 
                  'history of presenting complaint', 'history of the presenting complaint', 
                  'history of present complaint', 'history of the present complaint', 'interval history']
    historyHeaders = ['medical history', 'past medical history', 'surgical history', 
                      'past medical / surgical history', 'past surgical history']
    fhxHeaders = ['family hisotyr', 'fh', 'fhx']
    shxHeaders = ['social history', 'shx', 'family history / social history', 'social h?i?s?t?o?r?y? ?and family history', 
                  'family h?i?s?t?o?r?y? ?and social history']
    rosHeaders = ['review of systems', 'ros']
    peHeaders = ['physical examination', 'physical exam', 'examination', 'exam']
    allergyHeaders = ['allergy', 'allergies', 'adverse reaction']
    medHeaders = ['medication', 'current medication', 'current med', 'home med', 'home medication', 
                   'medications? at home', 'medications? at home', 'meds? at home', 'past medication',
                   'medications? on admission', 'medications? on admission', 'meds? on admission', 
                   'active medication', 'medications? \(?confirmed\)?', '\nmed']
    labHeaders = ['lab', 'laboratory', 'laboratories', 'laboratory data', 'lab evaluation', 
                  'laboratory evaluation', 'data', 'selected [a-z ]*lab[oratryies]*']
    radHeaders = ['radiological study', 'radiological studies', 'radiology', 'radiological exam', 
                  'radiological examination', 'diagnostic studies', 'studies', 'radiology of chest', 
                  'computed tomography', 'chest computed tomography', 'abdomen computed tomography',
                  'magnetic resonance imaging' 'echo', 'electrocardiogram']
    problemHeaders = ['problem', 'problem list', 'major problem', 'final diagnos[ie]s']
    assessmentHeaders = ['impression', 'assessment', 'assessment and plan', 'assessment / plan', 
                         'assessment & plan', 'a / p']
    planHeaders = ['plan', 'recommendation']
    
    headersList = (ccHeaders + hpiHeaders + historyHeaders + fhxHeaders + shxHeaders + rosHeaders + peHeaders 
                   + allergyHeaders + medHeaders + labHeaders + radHeaders + problemHeaders 
                   + assessmentHeaders + planHeaders)
    
    for dDays, dDic in preproc_dict.items():
    # standardize headers to "{headWord} :\n"
        for headWord in headersList:
            dDic['rtext'] = re.sub(rf'\n[^\.\n]*({headWord}s? :) (.+)', '\n\\1\n\\2', dDic['rtext'], flags=re.IGNORECASE) # plan : blah
            dDic['rtext'] = re.sub(rf'\.[^\.\n]*({headWord}s? :) (.+)', '\n\\1\n\\2', dDic['rtext'], flags=re.IGNORECASE) # plan : blah
            dDic['rtext'] = re.sub(rf'\n *({headWord}s?)\n', '\n\\1 :\n', dDic['rtext'], flags=re.IGNORECASE) # plan\nblah
            dDic['rtext'] = re.sub(rf'\. *({headWord}s?)\n', '\n\\1 :\n', dDic['rtext'], flags=re.IGNORECASE) # plan\nblah
            dDic['rtext'] = re.sub(rf'\n *({headWord}s?) \.', '\n\\1 :\n', dDic['rtext'], flags=re.IGNORECASE) # plan . blah
            dDic['rtext'] = re.sub(rf'\. *({headWord}s?) \.', '.\n\\1 :\n', dDic['rtext'], flags=re.IGNORECASE) # plan . blah
            dDic['rtext'] = re.sub('\n\n', '\n', dDic['rtext'])
            dDic['rtext'] = re.sub('\n ', '\n', dDic['rtext'])
        
        # standize header terms
        c = 0
        for ccHeader in ccHeaders:
            dDic['rtext'] = re.sub(rf'\n{ccHeader}s? :\n', rf'\nchief complaint{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for hpiHeader in hpiHeaders:
            dDic['rtext'] = re.sub(rf'\n{hpiHeader}s? :\n', rf'\nhistory of present illness{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for historyHeader in historyHeaders:
            dDic['rtext'] = re.sub(rf'\n{historyHeader}s? :\n', rf'\npast medical history{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for fhxHeader in fhxHeaders:
            dDic['rtext'] = re.sub(rf'\n{fhxHeader}s? :\n', rf'\nfamily history{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for shxHeader in shxHeaders:
            dDic['rtext'] = re.sub(rf'\n{shxHeader}s? :\n', rf'\nsocial history{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for rosHeader in rosHeaders:
            dDic['rtext'] = re.sub(rf'\n{rosHeader}s? :\n', rf'\nreview of systems{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for peHeader in peHeaders:
            dDic['rtext'] = re.sub(rf'\n{peHeader}s? :\n', rf'\nphysical examination{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for allergyHeader in allergyHeaders:
            dDic['rtext'] = re.sub(rf'\n{allergyHeader}s? :\n', rf'\nallergies{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for medHeader in medHeaders:
            dDic['rtext'] = re.sub(rf'\n{medHeader}s? :\n', rf'\nmedications{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for labHeader in labHeaders:
            dDic['rtext'] = re.sub(rf'\n{labHeader}s? :\n', rf'\nlaboratory examinations{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for radHeader in radHeaders:
            dDic['rtext'] = re.sub(rf'\n{radHeader}s? :\n', rf'\nradiological examinations{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for problemHeader in problemHeaders:
            dDic['rtext'] = re.sub(rf'\n{problemHeader}s? :\n', rf'\nproblem list{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for assessmentHeader in assessmentHeaders:
            dDic['rtext'] = re.sub(rf'\n{assessmentHeader}s? :\n', rf'\nassessment{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1
        for planHeader in planHeaders:
            dDic['rtext'] = re.sub(rf'\n{planHeader}s? :\n', rf'\nplans{c} :\n', dDic['rtext'], count=1, flags=re.IGNORECASE)
            c+=1

        # preproc_dict[dDays] holds subdictionaries for important sections like 'history of present illness' as well as 'text'
        # In order to build a subdictionary whose keys are important section headers and their values are corresonding texts,
        # we first build the dictionary 'headingDic' whose keys are enumerating numbers (in order of appearnace)
        # and their values are section headers.
    

        headingDic = {}
        h = 0
        #print('dDays:', dDays)
        #print('dDic text:', dDic['text'])
        for heading in re.findall('[A-Za-z/ ]+\d{1,3} :\n', dDic['rtext']):
            if ('chief complaint' in heading or 'history of present illness' in heading 
                or 'past medical history' in heading or 'family history' in heading or 'social history' in heading
                or 'review of systems' in heading
                or 'physical examination' in heading or 'medication' in heading or 'allerg' in heading 
                or 'laboratory' in heading or 'radiologic' in heading or 'problem list' in heading 
                or 'assessment' in heading or 'plan' in heading):
                headingDic[h] = heading
                h += 1
        # 'headingDic' looks like : {0: 'history of present illness0 :\n', 1: 'family history23 :\n',...}
        #print('headingDic of',str(dDays)+':', headingDic)
        for k in headingDic.keys():
            if k < len(headingDic)-1:
                f = find_between(preproc_dict[dDays]['rtext'], headingDic[k], headingDic[k+1])
                if 'chief complaint' in headingDic[k]:
                    preproc_dict[dDays].setdefault('chief complaint', []).append(str(f))
                elif 'history of present illness' in headingDic[k]:
                    preproc_dict[dDays].setdefault('history of present illness', []).append(str(f))
                elif 'past medical history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('past medical history', []).append(str(f))
                elif 'family history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('family history', []).append(str(f))
                elif 'social history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('social history', []).append(str(f))
                elif 'review of systems' in headingDic[k]:
                    preproc_dict[dDays].setdefault('review of systems', []).append(str(f))
                elif 'physical examination' in headingDic[k]:
                    preproc_dict[dDays].setdefault('physical examination', []).append(str(f))
                elif 'medication' in headingDic[k]:
                    preproc_dict[dDays].setdefault('medications', []).append(str(f))
                elif 'allerg' in headingDic[k]:
                    preproc_dict[dDays].setdefault('allergies', []).append(str(f))
                elif 'laboratory' in headingDic[k]:
                    preproc_dict[dDays].setdefault('laboratory examinations', []).append(str(f))
                elif 'radiolog' in headingDic[k]:
                    preproc_dict[dDays].setdefault('radiologic examinations', []).append(str(f))
                elif 'problem list' in headingDic[k]:
                    preproc_dict[dDays].setdefault('problem list', []).append(str(f))
                elif 'assessment' in headingDic[k]:
                    preproc_dict[dDays].setdefault('assessment', []).append(str(f))
                elif 'plan' in headingDic[k]:
                    preproc_dict[dDays].setdefault('plan', []).append(str(f))
                else:
                    preproc_dict[dDays].setdefault(headingDic[k], []).append(str(f))
                 
            else:
                f = find_between(preproc_dict[dDays]['rtext'], headingDic[k], 'This is the end of the day')
                if 'chief complaint' in headingDic[k]:
                    preproc_dict[dDays].setdefault('chief complaint', []).append(str(f))
                elif 'history of present illness' in headingDic[k]:
                    preproc_dict[dDays].setdefault('history of present illness', []).append(str(f))
                elif 'past medical history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('past medical history', []).append(str(f))
                elif 'family history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('family history', []).append(str(f))
                elif 'social history' in headingDic[k]:
                    preproc_dict[dDays].setdefault('social history', []).append(str(f))
                elif 'review of systems' in headingDic[k]:
                    preproc_dict[dDays].setdefault('review of systems', []).append(str(f))
                elif 'physical examination' in headingDic[k]:
                    preproc_dict[dDays].setdefault('physical examination', []).append(str(f))
                elif 'medication' in headingDic[k]:
                    preproc_dict[dDays].setdefault('medications', []).append(str(f))
                elif 'allerg' in headingDic[k]:
                    preproc_dict[dDays].setdefault('allergies', []).append(str(f))
                elif 'laboratory' in headingDic[k]:
                    preproc_dict[dDays].setdefault('laboratory examinations', []).append(str(f))
                elif 'radiolog' in headingDic[k]:
                    preproc_dict[dDays].setdefault('radiologic examinations', []).append(str(f))
                elif 'problem list' in headingDic[k]:
                    preproc_dict[dDays].setdefault('problem list', []).append(str(f))
                elif 'assessment' in headingDic[k]:
                    preproc_dict[dDays].setdefault('assessment', []).append(str(f))
                elif 'plan' in headingDic[k]:
                    preproc_dict[dDays].setdefault('plan', []).append(str(f))
                else:
                    preproc_dict[dDays].setdefault(headingDic[k], []).append(str(f))
        
        for hd in preproc_dict[dDays]:
            if hd != 'rtext':
                preproc_dict[dDays][hd] = ''.join(preproc_dict[dDays][hd])

    #Now we have a dictionary which looks like: 
    # {
    # 2131:
    #     {
    #      'rtext': '\n\n\past medical history...', 
    #      'text': '\n\nXXX...',
    #      'past medical history': 'hypertension, dmx20...',
    #      'social history': 'no smoking,...',
    #      ...
    #      },
    # 617:
    #     {
    #      'rtext': '\n\n\nCCU JAR Transfer Note...',
    #      'text': '\n\n\nCCU JAR Transfer Note...',
    #      'history of present illness': 'female is a 65 year old...',
    #       ...
    #      }
    # ...
    # } 
    
    return preproc_dict
    

In [31]:
def aui_freq_from_rasent(rasent):
    # rasent is the output of replace_aui(sent)
    auiNumDic = {}
    for aui in re.findall('A\d{7,8}', rasent):
        auiNumDic.setdefault(aui, 0)
        if aui in auiNumDic:
            auiNumDic[aui] += 1
    return auiNumDic

In [32]:
def aui_freq_from_preproc_dict_s(file, cc=True, hpi=True, pmh=True, fhx=False, shx=False, ros=False, pe=False, 
                                  med=True, allerg=False, lab=False, rad=True, prob=True, ass=True, plan=True):
    f = open(file,'r')
    record = f.read()
    preprocDic = make_preproc_dict(record)
    
    auiFreqDic = {}
    
    for day in preprocDic.keys():
        print(' processing', day)
        if cc==True:
            try:
                ccSection = preprocDic[day]['chief complaint']
                #print('  processing cc')
                for s in listize(ccSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]                   
            except:
                pass
            
        if hpi==True:
            try:
                hpiSection = preprocDic[day]['history of present illness']
                #print('  processing hpi')
                for s in listize(hpiSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]  
            except:
                pass
            
        if pmh==True:
            try:
                pmhSection = preprocDic[day]['past medical history']
                #print('  processing pmh')
                for s in listize(pmhSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]  
            except:
                pass
            
        if fhx==True:
            try:
                fhxSection = preprocDic[day]['family history']
                #print(' processing fhx')
                for s in listize(fhxSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if shx==True:
            try:
                shxSection = preprocDic[day]['social history']
                #print('  processing shx')
                for s in listize(shxSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if ros==True:
            try: 
                rosSection = preprocDic[day]['review of systems']
                #print('  processing ros')
                for s in listize(rosSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass

        if pe==True:
            try: 
                peSection = preprocDic[day]['physical examination']
                #print('  processing pe')
                for s in listize(peSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if med==True:
            try: 
                medSection = preprocDic[day]['medications']
                #print('  processing med')
                for s in listize(medSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass

        if allerg==True:
            try: 
                allergSection = preprocDic[day]['allergies']
                #print('  processing allergies')
                for s in listize(allergSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if lab==True:
            try: 
                labSection = preprocDic[day]['laboratory examinations']
                #print('  processing lab')
                for s in listize(labSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
        
        if rad==True:
            try: 
                radSection = preprocDic[day]['radiologic examinations']
                #print('  processing rad')
                for s in listize(radSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
    
        if prob==True:
            try: 
                probSection = preprocDic[day]['problem list']
                #print('  processing problem')
                for s in listize(probSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if ass==True:
            try: 
                assSection = preprocDic[day]['assessment']
                #print('  processing assessment')
                for s in listize(assSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if plan==True:
            try: 
                planSection = preprocDic[day]['plan']
                #print('  processing plan')
                for s in listize(planSection):
                    raSent = replace_aui(s)
                    print(s)
                    print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
                                   
    return {k:v for k,v in sorted(auiFreqDic.items(), key=lambda item:item[1], reverse=True)}
                                

In [33]:
def aui_freq_from_preproc_dict(file, cc=True, hpi=True, pmh=True, fhx=False, shx=False, ros=False, pe=False, 
                                  med=True, allerg=False, lab=False, rad=True, prob=True, ass=True, plan=True):
    f = open(file,'r')
    record = f.read()
    preprocDic = make_preproc_dict(record)
    
    auiFreqDic = {}
    
    for day in preprocDic.keys():
        print(' processing', day)
        if cc==True:
            try:
                ccSection = preprocDic[day]['chief complaint']
                #print('  processing cc')
                for s in listize(ccSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]                   
            except:
                pass
            
        if hpi==True:
            try:
                hpiSection = preprocDic[day]['history of present illness']
                #print('  processing hpi')
                for s in listize(hpiSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]  
            except:
                pass
            
        if pmh==True:
            try:
                pmhSection = preprocDic[day]['past medical history']
                #print('  processing pmh')
                for s in listize(pmhSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui]  
            except:
                pass
            
        if fhx==True:
            try:
                fhxSection = preprocDic[day]['family history']
                #print(' processing fhx')
                for s in listize(fhxSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if shx==True:
            try:
                shxSection = preprocDic[day]['social history']
                #print('  processing shx')
                for s in listize(shxSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if ros==True:
            try: 
                rosSection = preprocDic[day]['review of systems']
                #print('  processing ros')
                for s in listize(rosSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass

        if pe==True:
            try: 
                peSection = preprocDic[day]['physical examination']
                #print('  processing pe')
                for s in listize(peSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if med==True:
            try: 
                medSection = preprocDic[day]['medications']
                #print('  processing med')
                for s in listize(medSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass

        if allerg==True:
            try: 
                allergSection = preprocDic[day]['allergies']
                #print('  processing allergies')
                for s in listize(allergSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if lab==True:
            try: 
                labSection = preprocDic[day]['laboratory examinations']
                #print('  processing lab')
                for s in listize(labSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
        
        if rad==True:
            try: 
                radSection = preprocDic[day]['radiologic examinations']
                #print('  processing rad')
                for s in listize(radSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
    
        if prob==True:
            try: 
                probSection = preprocDic[day]['problem list']
                #print('  processing problem')
                for s in listize(probSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if ass==True:
            try: 
                assSection = preprocDic[day]['assessment']
                #print('  processing assessment')
                for s in listize(assSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
            
        if plan==True:
            try: 
                planSection = preprocDic[day]['plan']
                #print('  processing plan')
                for s in listize(planSection):
                    raSent = replace_aui(s)
                    #print(s)
                    #print(raSent, '\n')
                    extAui = aui_freq_from_rasent(raSent)
                    negAui = negate(raSent)
                    for aui in extAui:
                        if aui not in negAui:
                            if aui not in auiFreqDic:
                                auiFreqDic.setdefault(aui, extAui[aui])
                            else:
                                auiFreqDic[aui] += extAui[aui] 
            except:
                pass
                                   
    return {k:v for k,v in sorted(auiFreqDic.items(), key=lambda item:item[1], reverse=True)}
                                

In [35]:
def aui_freq_from_most_recent_preproc_dict(file, cc=True, hpi=True, pmh=True, fhx=False, shx=False, ros=False, pe=False, 
                                  med=True, allerg=False, lab=False, rad=True, prob=True, ass=True, plan=True):
    f = open(file,'r')
    record = f.read()
    preprocDic = make_preproc_dict(record)
    
    auiFreqDic = {}
    
    for day in preprocDic.keys():
        if day==0:
            print(' processing', day)
            if cc==True:
                try:
                    ccSection = preprocDic[day]['chief complaint']
                    #print('  processing cc')
                    for s in listize(ccSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui]                   
                except:
                    pass

            if hpi==True:
                try:
                    hpiSection = preprocDic[day]['history of present illness']
                    #print('  processing hpi')
                    for s in listize(hpiSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui]  
                except:
                    pass

            if pmh==True:
                try:
                    pmhSection = preprocDic[day]['past medical history']
                    #print('  processing pmh')
                    for s in listize(pmhSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui]  
                except:
                    pass

            if fhx==True:
                try:
                    fhxSection = preprocDic[day]['family history']
                    #print(' processing fhx')
                    for s in listize(fhxSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if shx==True:
                try:
                    shxSection = preprocDic[day]['social history']
                    #print('  processing shx')
                    for s in listize(shxSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if ros==True:
                try: 
                    rosSection = preprocDic[day]['review of systems']
                    #print('  processing ros')
                    for s in listize(rosSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if pe==True:
                try: 
                    peSection = preprocDic[day]['physical examination']
                    #print('  processing pe')
                    for s in listize(peSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if med==True:
                try: 
                    medSection = preprocDic[day]['medications']
                    #print('  processing med')
                    for s in listize(medSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if allerg==True:
                try: 
                    allergSection = preprocDic[day]['allergies']
                    #print('  processing allergies')
                    for s in listize(allergSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if lab==True:
                try: 
                    labSection = preprocDic[day]['laboratory examinations']
                    #print('  processing lab')
                    for s in listize(labSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if rad==True:
                try: 
                    radSection = preprocDic[day]['radiologic examinations']
                    #print('  processing rad')
                    for s in listize(radSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if prob==True:
                try: 
                    probSection = preprocDic[day]['problem list']
                    #print('  processing problem')
                    for s in listize(probSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if ass==True:
                try: 
                    assSection = preprocDic[day]['assessment']
                    #print('  processing assessment')
                    for s in listize(assSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass

            if plan==True:
                try: 
                    planSection = preprocDic[day]['plan']
                    #print('  processing plan')
                    for s in listize(planSection):
                        raSent = replace_aui(s)
                        #print(s)
                        #print(raSent, '\n')
                        extAui = aui_freq_from_rasent(raSent)
                        negAui = negate(raSent)
                        for aui in extAui:
                            if aui not in negAui:
                                if aui not in auiFreqDic:
                                    auiFreqDic.setdefault(aui, extAui[aui])
                                else:
                                    auiFreqDic[aui] += extAui[aui] 
                except:
                    pass
        else:
            pass
                                   
    return {k:v for k,v in sorted(auiFreqDic.items(), key=lambda item:item[1], reverse=True)}
                                

# Exploiting SNOMED's hierarchical structure

In [37]:
# Do not use this function for root concept 'A3684559'
def direct_subconcepts(concept): # returns a list of immediate children of the concept
    sql1 = "select distinct ptr from mrhier where aui = %s and rela = %s"
    val1 = (concept, 'isa')
    mycursor.execute(sql1, val1)
    myresult = mycursor.fetchall()
    ptrList = list(set([i[0] for i in myresult]))
    chl = []
    try:
        for ptrs in ptrList:
            ptr = ptrs+'.'+concept
            #print(ptr)
            sql2 = "select distinct aui from mrhier where ptr = %s and rela = %s"
            val2 = (ptr,'isa')
            mycursor.execute(sql2,val2)
            myresult2 = mycursor.fetchall()
            children = []
            for aui in myresult2:
                aui = aui[0]
                children.append(aui)
            #print(children)
            if children not in chl:
                chl.append(children)
        return chl[0]
    except:
        return []
    

In [38]:
#https://stackoverflow.com/questions/42388155/python-recursion-in-appending-lists
def all_subconcepts(concept, leavesList = None): # returns a list of all concepts that the 'concept' subsumes including itself
    if leavesList ==None:
        leavesList = []
    children = direct_subconcepts(concept) # returns a list of subsuming concepts
    if len(children)==0:
        leavesList.append(concept)
    else:
        for child in children:
            leavesList.append(child)
            all_subconcepts(child, leavesList)
    leavesList.append(concept)
    return list(set(leavesList))

In [39]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [40]:
def all_superconcepts(aui):
    all_superconcepts = []
    for clist in show_ptr(aui):
        for a in clist:
            if a not in all_superconcepts:
                all_superconcepts.append(a)
            else:
                continue
    return all_superconcepts

In [42]:
def common_ancestors(aui1, aui2):
    hier1 = show_ptr(aui1)
    hier2 = show_ptr(aui2)
    comAnc = []
    for h in hier1:
        for i in hier2:
            inter = intersection(h, i)
            if inter not in comAnc:
                comAnc.append(inter)
    # the list comAnc is a list of list whose elements are lists of common ancesters
    # since SNOMED CT is multihierarchical, there may be multiple lists of common ancesters
    # ex. comAnc = [['A3684559', 'A3886745'], [A3684559', 'A3886745', 'A7843222']]
    return comAnc

# Similarity measures

In [48]:
def sim_ch(qaui, taui): # qaui is AUI of query concept; taui is AUI of concept in text
    depth_qaui = min([len(x) for x in show_ptr(qaui)])
    if show_ptr(taui):
        depth_taui = min([len(x) for x in show_ptr(taui)])

        scoreList = []
        for hier in show_ptr(taui):
            if qaui in hier:

                score = math.log((depth_taui/depth_qaui)+1)
                scoreList.append(score)
            else:
                score = 0
                scoreList.append(score)
        return max(scoreList)
    else:
        return 0

In [49]:
def sim_ch_2(qaui, taui): # qaui is AUI of query concept; taui is AUI of concept in text
    depth_qaui = min([len(x) for x in show_ptr(qaui)])
    if show_ptr(taui):
        depth_taui = min([len(x) for x in show_ptr(taui)])

        scoreList = []
        for hier in show_ptr(taui):
            if qaui in hier:
                score = (10-1/math.log(min(depth_qaui, depth_taui)))/10
                scoreList.append(score)
            else:
                score = 0
                scoreList.append(score)
        return max(scoreList)
    else:
        return 0

In [50]:
def sim_ch_ic(qaui, taui): # qaui is AUI of query concept; taui is AUI of concept in text
    depth_qaui = min([len(x) for x in show_ptr(qaui)])
    if show_ptr(taui):
        depth_taui = min([len(x) for x in show_ptr(taui)])

        scoreList = []
        for hier in show_ptr(taui):
            if qaui in hier:
                score = math.log((depth_qaui/depth_taui)+1)*(1/math.sqrt(len(all_subconcepts(taui))))
                scoreList.append(score)
            else:
                score = 0
                scoreList.append(score)
        return max(scoreList)
    else:
        return 0

# Calculate Similarity Score

## Major Diabetes

In [94]:
# Atomic AUIS extracted from free text
diabetes = ['A2928669', 'A2882662', 'A3032080', 'A3322678']
#common_diabetes = ['A2928669', 'A2882662', 'A3032080', 'A3322678']
nephropathy = ['A24807926', 'A23451959', 'A2890220', 'A3318512', 'A3063986'] 
#common_nephropathy = ['A2872555']
#neuropathy = ['A2961908', 'A2876139', 'A3282650', 'A3119481', 
#              'A2941987', 'A3274394', 'A3400640', 'A3162750', 'A2884524', 'A3199258', 'A2890156']
neuropathy = ['A2876139', 'A3282650', 'A3119481', 'A3400640', 'A3162750']
#retinopathy = ['A2928678','A3064437']
retinopathy = ['A3064437']
#skin_conditions = ['A3471880', 'A2881852', 'A3318513', 'A3282693', 'A3299809', 'A3769261'] 
skin_conditions = ['A3471880', 'A2881852', 'A3769261']
amputation = ['A3241080', 'A2878436', 'A2991260']
#common_amputation = amputation
queryAui = diabetes + nephropathy + neuropathy + neuropathy + retinopathy + skin_conditions + amputation
cx = nephropathy + neuropathy + neuropathy + retinopathy + skin_conditions + amputation

In [52]:
in_dir = './n2c2-2018-t1/testbed/'
out = open('diabetes-17-v7.txt', 'w')
out.write('Num,Score,Count\n')
out.close()

In [53]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('diabetes-17-v7.txt', 'a')
        #out = open('major_diabetes_scores.txt', 'a')
        
        doc = aui_freq_from_preproc_dict(in_dir + file)
        
        count = 0
        scoreDic = {}
        irrelev = ['A3092358', 'A18073061', 'A24256890', 'A3665747', 'A2953455', 'A2967862', 'A2972962', 'A26587941', 'A24246786', # nephropathy 
                   'A3318307', 'A3322744', 'A3121005', 'A3034272', 'A6917003', 'A3580981', 'A3318879', 'A3162763', 'A2976161', 'A2874833', # neuropathy
                   'A15154924', 'A3367488', 'A3307688', 'A3019101', 'A2892313', 'A8387117', 'A2891853', 'A3030153', 'A2890283', 'A3034481', 'A2936406', 'A3578402', 'A2939246', 'A3904017', 'A3238102', 'A2892298', 'A7873926', 'A3070798', 'A11734429'] # retinoapathy
        for qaui in queryAui:
            for daui in doc:
                if intersection(irrelev, all_superconcepts(daui)):
                    pass
                else:
                    score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                    if score > 0:
                        print('Score between', qaui, 'and', daui, 'is', score)
                        count += doc[daui]
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
                    else:
                        continue
                        
        if intersection(scoreDic, diabetes):
            if intersection(scoreDic, cx):
                totalScore = sum(scoreDic.values())
            else:
                totalScore = 0
        else:
            totalScore = 0
            
        #out.write(fileNum+','+str(totalScore)+'\n')
        out.write(fileNum+','+str(totalScore)+','+str(count)+'\n')
        print('Score of', file+':', totalScore)
        print('Count of', file+':', count)
        print('**********************************************************************')

Processing file 360.xml ......
 processing 899
 processing 788
 processing 635
 processing 0
Score of 360.xml: 0
Count of 360.xml: 0
**********************************************************************
Processing file 348.xml ......
 processing 1442
 processing 1264
 processing 1147
 processing 594
 processing 0
Score between A2928669 and A23027694 is 0.2654751135014336
Score of 348.xml: 0
Count of 348.xml: 5
**********************************************************************
Processing file 361.xml ......
 processing 1481
 processing 1254
 processing 789
 processing 13
 processing 0
Score between A2928669 and A2928669 is 0.1332536905487145
Score between A2928669 and A22857142 is 0.24537489466018364
Score between A2928669 and A23027694 is 0.20539958581787626
Score between A3322678 and A2928678 is 0.07967362644124298
Score between A3322678 and A2961908 is 0.08460397891070054
Score between A3322678 and A26592686 is 0.2352255879372843
Score between A24807926 and A15118214 is 0.492573

 processing 335
 processing 278
 processing 139
 processing 0
Score between A2928669 and A2928669 is 0.1332536905487145
Score between A2928669 and A23027694 is 0.20539958581787626
Score between A24807926 and A15118214 is 0.41010613169153837
Score between A24807926 and A23451959 is 0.5552863273064585
Score between A24807926 and A24807926 is 0.14535960673210757
Score between A24807926 and A3481042 is 0.6148007815585422
Score between A24807926 and A15118845 is 0.13568868014581206
Score between A23451959 and A23451959 is 0.6217621548833854
Score between A23451959 and A3481042 is 0.6905971980046067
Score between A2890220 and A2890220 is 0.3807500052094045
Score between A3063986 and A15118214 is 0.5049087429483703
Score between A3063986 and A2890220 is 0.32287482658835615
Score between A3063986 and A23451959 is 0.6836496697581711
Score between A3063986 and A24807926 is 0.17768648163777476
Score between A3063986 and A3481042 is 0.761500010418809
Score between A3063986 and A15118845 is 0.16705

Score between A2928669 and A23027694 is 0.10269979290893813
Score of 293.xml: 0
Count of 293.xml: 9
**********************************************************************
Processing file 251.xml ......
 processing 276
 processing 140
 processing 70
 processing 0
Score between A2928669 and A2928669 is 0.088835793699143
Score between A2928669 and A23027694 is 0.10269979290893813
Score between A24807926 and A24807926 is 0.05623277192282478
Score between A3063986 and A24807926 is 0.06873851422920199
Score of 251.xml: 0.3165068727601079
Count of 251.xml: 6
**********************************************************************
Processing file 327.xml ......
 processing 715
 processing 399
 processing 225
 processing 0
Score between A2928669 and A2928669 is 0.17353586347830016
Score between A3063986 and A2979587 is 0.04672829151860525
Score of 327.xml: 0.2202641549969054
Count of 327.xml: 15
**********************************************************************
Processing file 285.xml ......


Score between A2928669 and A22857142 is 0.16749442306638748
Score between A2928669 and A23027694 is 0.16277532059249547
Score between A2882662 and A3059581 is 0.4357183199541855
Score between A3064437 and A3064437 is 0.017159036211364335
Score of 352.xml: 0.9239485015593752
Count of 352.xml: 14
**********************************************************************
Processing file 350.xml ......
 processing 869
 processing 680
 processing 592
 processing 0
Score between A2928669 and A2928669 is 0.1332536905487145
Score between A2876139 and A2890242 is 0.3503527057713633
Score between A3400640 and A2890242 is 0.3503527057713633
Score between A2876139 and A2890242 is 0.3503527057713633
Score between A3400640 and A2890242 is 0.3503527057713633
Score of 350.xml: 1.5346645136341677
Count of 350.xml: 11
**********************************************************************
Processing file 386.xml ......
 processing 1737
 processing 585
 processing 374
 processing 0
Score between A2928669 and 

In [54]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('diabetes-17-v7.txt', 'a')
        #out = open('major_diabetes_scores.txt', 'a')
        
        doc = aui_freq_from_preproc_dict(in_dir + file)
        
        count = 0
        scoreDic = {}
        irrelev = ['A3092358', 'A18073061', 'A24256890', 'A3665747', 'A2953455', 'A2967862', 'A2972962', 'A26587941', 'A24246786', # nephropathy 
                   'A3318307', 'A3322744', 'A3121005', 'A3034272', 'A6917003', 'A3580981', 'A3318879', 'A3162763', 'A2976161', 'A2874833', # neuropathy
                   'A15154924', 'A3367488', 'A3307688', 'A3019101', 'A2892313', 'A8387117', 'A2891853', 'A3030153', 'A2890283', 'A3034481', 'A2936406', 'A3578402', 'A2939246', 'A3904017', 'A3238102', 'A2892298', 'A7873926', 'A3070798', 'A11734429'] # retinoapathy
        for qaui in queryAui:
            for daui in doc:
                if intersection(irrelev, all_superconcepts(daui)):
                    pass
                else:
                    score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                    if score > 0:
                        print('Score between', qaui, 'and', daui, 'is', score)
                        count += doc[daui]
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
                    else:
                        continue
                        
        if intersection(scoreDic, diabetes):
            if intersection(scoreDic, cx):
                totalScore = sum(scoreDic.values())
            else:
                totalScore = 0
        else:
            totalScore = 0
            
        #out.write(fileNum+','+str(totalScore)+'\n')
        out.write(fileNum+','+str(totalScore)+','+str(count)+'\n')
        print('Score of', file+':', totalScore)
        print('Count of', file+':', count)
        print('**********************************************************************')

Processing file 229.xml ......
 processing 940
 processing 234
 processing 0
Score between A2928669 and A2928669 is 0.0444178968495715
Score of 229.xml: 0
Count of 229.xml: 1
**********************************************************************
Processing file 201.xml ......
 processing 2260
 processing 574
 processing 443
 processing 171
 processing 0
Score between A2928669 and A2928669 is 0.18155650285911404
Score between A2928669 and A23027694 is 0.10269979290893813
Score between A24807926 and A24807926 is 0.11246554384564957
Score between A3063986 and A24807926 is 0.13747702845840398
Score between A2878436 and A2900934 is 0.11512711783407903
Score of 201.xml: 0.6493259859061847
Count of 201.xml: 24
**********************************************************************
Processing file 217.xml ......
 processing 1999
 processing 1678
 processing 1471
 processing 0
Score between A2928669 and A2928669 is 0.07040070086747122
Score between A2928669 and A23027694 is 0.10269979290893813
S

Score of 115.xml: 0
Count of 115.xml: 4
**********************************************************************
Processing file 120.xml ......
 processing 1762
 processing 1273
 processing 1233
 processing 992
 processing 0
Score between A2928669 and A2928669 is 0.1332536905487145
Score between A2928669 and A23027694 is 0.10269979290893813
Score between A2882662 and A2882662 is 0.07898598893595904
Score between A3032080 and A3032080 is 0.0942245882051298
Score of 120.xml: 0
Count of 120.xml: 10
**********************************************************************
Processing file 108.xml ......
 processing 60
 processing 7
 processing 4
 processing 0
Score between A2928669 and A2928669 is 0.1246968013480915
Score between A2928669 and A23027694 is 0.23846153449437735
Score of 108.xml: 0
Count of 108.xml: 10
**********************************************************************
Processing file 135.xml ......
 processing 991
 processing 900
 processing 770
 processing 0
Score between A3064

Score between A3282650 and A3282650 is 0.4804530139182014
Score between A3400640 and A3400640 is 0.02323634269524533
Score between A3400640 and A3274394 is 0.028251818207656536
Score between A3400640 and A11738581 is 0.09246322563954687
Score between A3400640 and A3327123 is 0.07898598893595904
Score between A3400640 and A3319738 is 0.12295178163299307
Score between A3400640 and A2941987 is 0.055851528395342256
Score between A2876139 and A2876139 is 0.026220617272778874
Score between A2876139 and A3274394 is 0.028251818207656536
Score between A2876139 and A11738581 is 0.09246322563954687
Score between A2876139 and A3327123 is 0.07898598893595904
Score between A2876139 and A3319738 is 0.12295178163299307
Score between A2876139 and A2941987 is 0.055851528395342256
Score between A3282650 and A3282650 is 0.4804530139182014
Score between A3400640 and A3400640 is 0.02323634269524533
Score between A3400640 and A3274394 is 0.028251818207656536
Score between A3400640 and A11738581 is 0.09246322

## ABDOMINAL

In [51]:
in_dir = './n2c2-2018-t1/testbed/'

query = ['A3599911', 'A3709305', 'A3532649', 'A3071416']
irrelev = ['A3600839', 'A3124692', 'A3089277']

In [52]:
out = open('abdominal-17-v5.txt', 'w')
out.write('Num,Score,Count\n')
out.close()

In [59]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('abdominal-17-v5.txt', 'a')
        doc = aui_freq_from_preproc_dict(in_dir + file, fhx=True, shx=True, ros=True, pe=True, allerg=True, lab=True)
        
        scoreDic = {}
        count = 0
        for qaui in query:
            for daui in doc:
                try:
                    score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                    if score > 0:
                        if intersection(all_superconcepts(daui), irrelev):
                            pass
                        else:
                            print('Score between', qaui, 'and', daui, 'is', score)
                            count += 1
                            if qaui not in scoreDic:
                                scoreDic.setdefault(qaui, score)
                            else:
                                scoreDic[qaui] += score
                    else:
                        continue
                except:
                    pass
        
        totalScore = sum(scoreDic.values())
            
        #out.write(fileNum+','+str(totalScore)+'\n')
        print('Score of', file+':', totalScore, '\n')
        print('Number of relevant concepts in', file+':', count, '\n')
        out.write(fileNum+','+str(totalScore)+'\n')

Processing file 229.xml ......
 processing 940
 processing 234
 processing 0
Score of 229.xml: 0 

Number of relevant concepts in 229.xml: 0 

Processing file 201.xml ......
 processing 2260
 processing 574
 processing 443
 processing 171
 processing 0
Score of 201.xml: 0 

Number of relevant concepts in 201.xml: 0 

Processing file 217.xml ......
 processing 1999
 processing 1678
 processing 1471
 processing 0
Score of 217.xml: 0 

Number of relevant concepts in 217.xml: 0 

Processing file 216.xml ......
 processing 2969
 processing 757
 processing 662
 processing 422
 processing 0
Score between A3599911 and A2882818 is 0.03389064010294695
Score of 216.xml: 0.03389064010294695 

Number of relevant concepts in 216.xml: 1 

Processing file 158.xml ......
 processing 3707
 processing 3253
 processing 1225
 processing 0
Score between A3599911 and A2878619 is 0.14321021207541346
Score between A3532649 and A2878619 is 0.179098729664748
Score of 158.xml: 0.32230894174016145 

Number of rele

 processing 760
 processing 714
 processing 551
 processing 0
Score of 153.xml: 0 

Number of relevant concepts in 153.xml: 0 



In [60]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('abdominal-17-v5.txt', 'a')
        doc = aui_freq_from_preproc_dict(in_dir + file, fhx=True, shx=True, ros=True, pe=True, allerg=True, lab=True)
        
        scoreDic = {}
        count = 0
        for qaui in query:
            for daui in doc:
                try:
                    score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                    if score > 0:
                        if intersection(all_superconcepts(daui), irrelev):
                            pass
                        else:
                            print('Score between', qaui, 'and', daui, 'is', score)
                            count += 1
                            if qaui not in scoreDic:
                                scoreDic.setdefault(qaui, score)
                            else:
                                scoreDic[qaui] += score
                    else:
                        continue
                except:
                    pass
        
        totalScore = sum(scoreDic.values())
            
        #out.write(fileNum+','+str(totalScore)+'\n')
        print('Score of', file+':', totalScore, '\n')
        print('Number of relevant concepts in', file+':', count, '\n')
        out.write(fileNum+','+str(totalScore)+'\n')

Processing file 360.xml ......
 processing 899
 processing 788
 processing 635
 processing 0
Score of 360.xml: 0 

Number of relevant concepts in 360.xml: 0 

Processing file 348.xml ......
 processing 1442
 processing 1264
 processing 1147
 processing 594
 processing 0
Score of 348.xml: 0 

Number of relevant concepts in 348.xml: 0 

Processing file 361.xml ......
 processing 1481
 processing 1254
 processing 789
 processing 13
 processing 0
Score between A3599911 and A2879848 is 0.17175273835878474
Score of 361.xml: 0.17175273835878474 

Number of relevant concepts in 361.xml: 1 

Processing file 399.xml ......
 processing 764
 processing 609
 processing 381
 processing 0
Score of 399.xml: 0 

Number of relevant concepts in 399.xml: 0 

Processing file 373.xml ......
 processing 2271
 processing 1293
 processing 183
 processing 34
 processing 0
Score of 373.xml: 0 

Number of relevant concepts in 373.xml: 0 

Processing file 359.xml ......
 processing 1789
 processing 1295
 processin

 processing 1631
 processing 1014
 processing 744
 processing 0
Score of 353.xml: 0 

Number of relevant concepts in 353.xml: 0 

Processing file 347.xml ......
 processing 1027
 processing 917
 processing 195
 processing 0
Score of 347.xml: 0 

Number of relevant concepts in 347.xml: 0 

Processing file 384.xml ......
 processing 3598
 processing 1301
 processing 585
 processing 0
Score of 384.xml: 0 

Number of relevant concepts in 384.xml: 0 

Processing file 346.xml ......
 processing 825
 processing 775
 processing 13
 processing 0
Score of 346.xml: 0 

Number of relevant concepts in 346.xml: 0 

Processing file 352.xml ......
 processing 3753
 processing 945
 processing 882
 processing 542
 processing 0
Score between A3599911 and A2879848 is 0.22198719402681533
Score of 352.xml: 0.22198719402681533 

Number of relevant concepts in 352.xml: 1 

Processing file 350.xml ......
 processing 869
 processing 680
 processing 592
 processing 0
Score of 350.xml: 0 

Number of relevant conc

## ADVANCED-CAD

In [68]:
#mis = ['A17310474', 'A7873496', 'A7876887', 'A2886848', 'A2938836', 'A24246836', 'A3461772', 'A26575074', 'A6919959']
mis = ['A7873496', 'A2938836', 'A24246836', 'A26575074']
#mis = ['A2886848', 'A2938836', 'A24246836', 'A3461772', 'A26575074', 'A6919959'] # - stent
anginas = ['A23468083', 'A3313085']
#ischemias = ['A7873407']

nitrates = ['A2872933', 'A2884643']
antiplatelets = ['A3651057']
anticoagulates = ['A2882344']
statins = ['A3483678']
aceis = ['A3609517']
arbs = ['A3249419']
bbs = ['A3802645']
ccbs = ['A3335705']

cad_meds = nitrates+antiplatelets+anticoagulates+statins+aceis+arbs+bbs+ccbs
query = cad_meds + mis

In [69]:
in_dir = './n2c2-2018-t1/testbed/'
out = open('advanced_cad-17-v6.txt', 'w')
out.write('Num,Score,Count\n')
out.close()

In [70]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('advanced_cad-17-v6.txt', 'a')
        #out = open('major_diabetes_scores.txt', 'a')
        
        doc = aui_freq_from_preproc_dict(in_dir + file)
        day0doc = aui_freq_from_most_recent_preproc_dict(in_dir+file)
        
        catBin = {'cad_meds':False, 'mis':False, 'anginas':False, 'ischemias':False}
        numCat = 0
        numCadMeds = 0
        scoreDic = {}
        count = 0
        
        # examine cad_med, mis, and ischemias items
        for qaui in query:
            for daui in doc:    
                score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                if score > 0:
                    if qaui in cad_meds: # taking 2 or more medications to treat CAD     
                        numCadMeds += 1
                        print("There is cad_med:", daui, '*', doc[daui])
                        count += doc[daui]
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
                    elif qaui in mis: # a history of MI
                        print("There is mi:", daui, '*', doc[daui])
                        count += doc[daui]
                        catBin['mis'] = True
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
            
                else:
                    continue
        # examine anginas items
        # consider the most recent patient record only by using 'day0doc'
        for angaui in anginas:
            for d0aui in day0doc:
                score = sim_ch_ic(angaui, d0aui)*math.log(day0doc[d0aui]+1)
                if score > 0:
                    if angaui in anginas: # taking 2 or more medications to treat CAD
                        print("There is angina:", d0aui, '*', day0doc[d0aui])
                        count += day0doc[d0aui]
                        catBin['anginas'] = True
                        if angaui not in scoreDic:
                            scoreDic.setdefault(angaui, score)
                        else:
                            scoreDic[angaui] += score
                            
        if numCadMeds >= 2:
            catBin['cad_meds'] = True
        else:
            pass

        for v in catBin.values():
            if v==True:
                numCat+=1
            else:
                numCat+=0
        
        if numCat >= 2:
            totalScore = sum(scoreDic.values())
        else:
            totalScore = 0
            
        out.write(fileNum+','+str(totalScore)+','+str(count)+'\n')
        print('Score of', file+':', totalScore)
        print('Count of', file+':', count)
        print('**********************************************************************')
        print()

Processing file 229.xml ......
 processing 940
 processing 234
 processing 0
 processing 0
There is cad_med: A2884643 * 5
There is cad_med: A2878765 * 1
There is cad_med: A6936998 * 1
There is cad_med: A3735899 * 1
There is cad_med: A2888483 * 1
There is cad_med: A3299967 * 1
There is mi: A7873496 * 2
There is mi: A17310474 * 2
There is mi: A2938836 * 3
There is mi: A3051165 * 1
There is mi: A24246836 * 1
There is mi: A26575074 * 3
There is angina: A23468083 * 1
There is angina: A3313085 * 1
Score of 229.xml: 5.147536749389003
Count of 229.xml: 24
**********************************************************************

Processing file 201.xml ......
 processing 2260
 processing 574
 processing 443
 processing 171
 processing 0
 processing 0
There is cad_med: A2884643 * 1
There is cad_med: A2878765 * 8
There is cad_med: A3436370 * 2
There is cad_med: A3214864 * 5
There is cad_med: A6946482 * 5
There is cad_med: A2882344 * 5
There is cad_med: A3013150 * 1
There is cad_med: A3071038 * 1
Th

There is cad_med: A2884069 * 2
There is cad_med: A3044848 * 1
There is cad_med: A3802645 * 1
There is mi: A7873496 * 3
There is mi: A17310474 * 3
There is mi: A2938836 * 2
There is mi: A26575074 * 2
Score of 108.xml: 5.655014713226197
Count of 108.xml: 31
**********************************************************************

Processing file 135.xml ......
 processing 991
 processing 900
 processing 770
 processing 0
 processing 0
There is cad_med: A2878765 * 4
There is cad_med: A6936998 * 1
There is cad_med: A3214864 * 1
There is cad_med: A6946482 * 1
There is cad_med: A2882344 * 1
There is cad_med: A6920068 * 1
There is cad_med: A3609517 * 1
There is cad_med: A2884069 * 4
There is cad_med: A2887943 * 5
There is cad_med: A3082323 * 2
Score of 135.xml: 0
Count of 135.xml: 21
**********************************************************************

Processing file 137.xml ......
 processing 907
 processing 651
 processing 40
 processing 0
 processing 0
There is cad_med: A2884643 * 7
There

 processing 2401
 processing 2306
 processing 1730
 processing 0
 processing 0
There is cad_med: A2888483 * 1
There is cad_med: A2880719 * 7
Score of 192.xml: 0
Count of 192.xml: 8
**********************************************************************

Processing file 227.xml ......
 processing 1111
 processing 945
 processing 546
 processing 167
 processing 0
 processing 0
There is cad_med: A2884643 * 2
There is cad_med: A2878765 * 9
There is cad_med: A3060093 * 4
There is cad_med: A3735899 * 1
There is cad_med: A3071038 * 1
There is cad_med: A2885744 * 1
There is cad_med: A2888483 * 1
There is cad_med: A3299967 * 3
There is cad_med: A2878798 * 3
There is cad_med: A3004859 * 3
There is mi: A7873496 * 6
There is mi: A17310474 * 6
Score of 227.xml: 7.040290761184849
Count of 227.xml: 40
**********************************************************************

Processing file 233.xml ......
 processing 2814
 processing 2525
 processing 2291
 processing 1726
 processing 0
 processing 0
Ther

In [71]:
for file in os.listdir(in_dir):
    # --- read document
    if file.endswith('xml'):
        string = str(file)
        fileNum = re.findall('([0-9]{3})\.xml',string)[0]    
        print('Processing file', file, '......')
        out = open('advanced_cad-17-v6.txt', 'a')
        #out = open('major_diabetes_scores.txt', 'a')
        
        doc = aui_freq_from_preproc_dict(in_dir + file)
        day0doc = aui_freq_from_most_recent_preproc_dict(in_dir+file)
        
        catBin = {'cad_meds':False, 'mis':False, 'anginas':False, 'ischemias':False}
        numCat = 0
        numCadMeds = 0
        scoreDic = {}
        count = 0
        
        # examine cad_med, mis, and ischemias items
        for qaui in query:
            for daui in doc:    
                score = sim_ch_ic(qaui, daui)*math.log(doc[daui]+1)
                if score > 0:
                    if qaui in cad_meds: # taking 2 or more medications to treat CAD     
                        numCadMeds += 1
                        print("There is cad_med:", daui, '*', doc[daui])
                        count += doc[daui]
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
                    elif qaui in mis: # a history of MI
                        print("There is mi:", daui, '*', doc[daui])
                        count += doc[daui]
                        catBin['mis'] = True
                        if qaui not in scoreDic:
                            scoreDic.setdefault(qaui, score)
                        else:
                            scoreDic[qaui] += score
            
                else:
                    continue
        # examine anginas items
        # consider the most recent patient record only by using 'day0doc'
        for angaui in anginas:
            for d0aui in day0doc:
                score = sim_ch_ic(angaui, d0aui)*math.log(day0doc[d0aui]+1)
                if score > 0:
                    if angaui in anginas: # taking 2 or more medications to treat CAD
                        print("There is angina:", d0aui, '*', day0doc[d0aui])
                        count += day0doc[d0aui]
                        catBin['anginas'] = True
                        if angaui not in scoreDic:
                            scoreDic.setdefault(angaui, score)
                        else:
                            scoreDic[angaui] += score
                            
        if numCadMeds >= 2:
            catBin['cad_meds'] = True
        else:
            pass

        for v in catBin.values():
            if v==True:
                numCat+=1
            else:
                numCat+=0
        
        if numCat >= 2:
            totalScore = sum(scoreDic.values())
        else:
            totalScore = 0
            
        out.write(fileNum+','+str(totalScore)+','+str(count)+'\n')
        print('Score of', file+':', totalScore)
        print('Count of', file+':', count)
        print('**********************************************************************')
        print()

Processing file 360.xml ......
 processing 899
 processing 788
 processing 635
 processing 0
 processing 0
There is cad_med: A3735899 * 3
There is cad_med: A3253741 * 1
There is cad_med: A2888483 * 5
There is cad_med: A2878798 * 5
Score of 360.xml: 0
Count of 360.xml: 14
**********************************************************************

Processing file 348.xml ......
 processing 1442
 processing 1264
 processing 1147
 processing 594
 processing 0
 processing 0
There is cad_med: A2878765 * 3
There is cad_med: A6936998 * 2
There is cad_med: A3313326 * 1
There is cad_med: A3735899 * 3
There is cad_med: A2888483 * 3
There is cad_med: A2878798 * 3
Score of 348.xml: 0
Count of 348.xml: 15
**********************************************************************

Processing file 361.xml ......
 processing 1481
 processing 1254
 processing 789
 processing 13
 processing 0
 processing 0
There is cad_med: A2884643 * 1
There is cad_med: A2878765 * 1
There is cad_med: A3214864 * 1
There is cad_m

There is cad_med: A2878765 * 7
There is cad_med: A3354508 * 1
There is cad_med: A3483678 * 2
There is cad_med: A3071038 * 1
There is cad_med: A3609517 * 2
There is cad_med: A2888483 * 1
There is cad_med: A3802645 * 2
There is cad_med: A3044848 * 1
There is cad_med: A13013175 * 1
There is mi: A7873496 * 10
There is mi: A17310474 * 10
Score of 294.xml: 4.261793046459139
Count of 294.xml: 38
**********************************************************************

Processing file 323.xml ......
 processing 2360
 processing 2038
 processing 661
 processing 420
 processing 0
 processing 0
There is cad_med: A2878765 * 1
There is cad_med: A3735899 * 2
There is cad_med: A3905549 * 2
There is cad_med: A3071038 * 1
There is cad_med: A3483678 * 1
There is cad_med: A2878798 * 1
There is cad_med: A2883275 * 1
There is cad_med: A13013175 * 2
Score of 323.xml: 0
Count of 323.xml: 11
**********************************************************************

Processing file 321.xml ......
 processing 1525
 

 processing 1631
 processing 1014
 processing 744
 processing 0
 processing 0
There is cad_med: A2878765 * 1
There is cad_med: A6936998 * 1
There is cad_med: A2888483 * 1
There is cad_med: A2878798 * 1
Score of 353.xml: 0
Count of 353.xml: 4
**********************************************************************

Processing file 347.xml ......
 processing 1027
 processing 917
 processing 195
 processing 0
 processing 0
There is cad_med: A2878765 * 3
There is cad_med: A3253741 * 1
There is cad_med: A2888483 * 2
There is cad_med: A2878798 * 2
There is cad_med: A3077321 * 1
There is cad_med: A2887393 * 1
There is cad_med: A3802645 * 1
There is cad_med: A2888239 * 1
Score of 347.xml: 0
Count of 347.xml: 12
**********************************************************************

Processing file 384.xml ......
 processing 3598
 processing 1301
 processing 585
 processing 0
 processing 0
There is cad_med: A3735899 * 1
There is cad_med: A3071038 * 1
There is cad_med: A2888483 * 1
There is cad_me